For now, Let's try to do CD using a simple Unet on SN7 

Later, try to pretarain Unet (or other model) on ONERA and compare


Later try it on other more advanced model


In [11]:
%pip install tabulate
%pip install fvcore
%pip install wandb

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # image plotting
import matplotlib
matplotlib.rcParams['figure.dpi'] = 300 #increase plot resolution

#Raster data handling
from PIL import Image
import skimage
from skimage import io, transform 
import rasterio as rio # geo-oriented plotting library
from rasterio import features
import cv2

#PyTorch
import torch
from torch.utils.data import Dataset, DataLoader, Sampler # custom dataset handling
import torch.autograd.profiler as profiler # to track model inference and detect leaks
from torchvision import transforms, utils
from torch.autograd import Variable
from torch import nn
from torch.nn.modules.padding import ReplicationPad2d
import torchvision.models as models
from torch import optim
from collections import OrderedDict
from torch.cuda.amp import GradScaler
from torch.cuda.amp import autocast
from torch import optim
from torch.utils import data as torch_data

#Augmentations
#import albumentations as alb
#from albumentations.pytorch import ToTensorV2
#Logging errors and progress, sending them to tg-bot

import requests
import traceback

#Other
from pathlib import Path # to have path strings as PosixPath objexts
import pathlib 
from pyproj import CRS
import geopandas as gpd # to make dataframes out of geojson files
import itertools
import re
from tqdm.notebook import tqdm
tqdm.pandas()
import os
import gc
from timeit import default_timer as time
import copy
import json
import logging
import datetime
import sys
import timeit

from tabulate import tabulate

/home/antonimestre/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [6]:
#Input
train_dir = Path('/home/antonimestre/Documents/Tesnime/SN7/SN7_buildings_train')
test_dir = Path('/home/antonimestre/Documents/Tesnime/SN7/SN7_buildings_test_public')
sample_dir = Path('/home/antonimestre/Documents/Tesnime/SN7/SN7_buildings_train_sample')

# Output
output_path = Path.cwd()
output_csv_path = output_path/'output_csvs/'
Path(output_csv_path).mkdir(parents=True, exist_ok=True)

test_raster_path = Path('/home/antonimestre/Documents/Tesnime/SN7/SN7_buildings_train_sample/sample/L15-0506E-1204N_2027_3374_13/images_masked/global_monthly_2018_01_mosaic_L15-0506E-1204N_2027_3374_13.tif')
test_raster_path_24 = Path('/home/antonimestre/Documents/Tesnime/SN7/SN7_buildings_train_sample/sample/L15-0506E-1204N_2027_3374_13/images_masked/global_monthly_2019_12_mosaic_L15-0506E-1204N_2027_3374_13.tif')
test_geojson_path = Path('/home/antonimestre/Documents/Tesnime/SN7/SN7_buildings_train_sample/sample/L15-0506E-1204N_2027_3374_13/labels_match_pix/global_monthly_2018_01_mosaic_L15-0506E-1204N_2027_3374_13_Buildings.geojson')
test_geojson_path_24 = Path('/home/antonimestre/Documents/Tesnime/SN7/SN7_buildings_train_sample/sample/L15-0506E-1204N_2027_3374_13/labels_match_pix/global_monthly_2019_12_mosaic_L15-0506E-1204N_2027_3374_13_Buildings.geojson')
test_gdf = gpd.read_file(test_geojson_path)
test_gdf_24 = gpd.read_file(test_geojson_path_24)

test_gdf.set_index('Id',inplace=True)
test_gdf_24.set_index('Id',inplace=True)

test_gdf.sort_index(inplace=True)
test_gdf_24.sort_index(inplace=True)

In [1]:
import wandb
from utils import networks, datasets, loss_functions, evaluation, experiment_manager

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


ImportError: cannot import name 'TypeAlias' from 'typing_extensions' (/usr/lib/python3/dist-packages/typing_extensions.py)

In [10]:
def run_training(cfg):
    run_config = {
        'CONFIG_NAME': cfg.NAME,
        'device': device,
        'epochs': cfg.TRAINER.EPOCHS,
        'learning rate': cfg.TRAINER.LR,
        'batch size': cfg.TRAINER.BATCH_SIZE,
    }
    table = {'run config name': run_config.keys(),
             ' ': run_config.values(),
             }
    print(tabulate(table, headers='keys', tablefmt="fancy_grid", ))

    net = networks.create_network(cfg)
    net.to(device)
    optimizer = optim.AdamW(net.parameters(), lr=cfg.TRAINER.LR, weight_decay=0.01)

    change_criterion = loss_functions.get_criterion(cfg.MODEL.LOSS_TYPE)
    sem_criterion = loss_functions.get_criterion(cfg.MODEL.LOSS_TYPE)
    change_consistency_criterion = loss_functions.get_criterion(cfg.CONSISTENCY_TRAINER.LOSS_TYPE)

    # reset the generators
    dataset = datasets.SpaceNet7CDDataset(cfg=cfg, run_type='training')
    print(dataset)

    dataloader_kwargs = {
        'batch_size': cfg.TRAINER.BATCH_SIZE,
        'num_workers': 0 if cfg.DEBUG else cfg.DATALOADER.NUM_WORKER,
        'shuffle': cfg.DATALOADER.SHUFFLE,
        'drop_last': True,
        'pin_memory': True,
    }
    dataloader = torch_data.DataLoader(dataset, **dataloader_kwargs)

    # unpacking cfg
    epochs = cfg.TRAINER.EPOCHS
    save_checkpoints = cfg.SAVE_CHECKPOINTS
    steps_per_epoch = len(dataloader)

    # tracking variables
    global_step = epoch_float = 0

    for epoch in range(1, epochs + 1):
        print(f'Starting epoch {epoch}/{epochs}.')

        start = timeit.default_timer()
        loss_set, sem_loss_set, change_loss_set, change_sem_loss_set, consistency_loss_set = [], [], [], [], []
        n_labeled, n_notlabeled = 0, 0

        for i, batch in enumerate(dataloader):

            net.train()
            optimizer.zero_grad()

            x_t1 = batch['x_t1'].to(device)
            x_t2 = batch['x_t2'].to(device)

            logits_change, logits_sem_t1, logits_sem_t2 = net(x_t1, x_t2)
            logits_change_sem = net.outc_sem_change(torch.cat((logits_sem_t1, logits_sem_t2), dim=1))
            y_pred_change_sem = torch.sigmoid(logits_change_sem)

            supervised_loss, consistency_loss = None, None

            is_labeled = batch['is_labeled']
            n_labeled += torch.sum(is_labeled).item()
            if is_labeled.any():
                # change detection
                gt_change = batch['y_change'].to(device)
                change_loss = change_criterion(logits_change[is_labeled, ], gt_change[is_labeled, ])

                # semantic segmentation
                gt_sem_t1 = batch['y_sem_t1'].to(device)
                gt_sem_t2 = batch['y_sem_t2'].to(device)

                sem_t1_loss = sem_criterion(logits_sem_t1[is_labeled, ], gt_sem_t1[is_labeled, ])
                sem_t2_loss = sem_criterion(logits_sem_t2[is_labeled, ], gt_sem_t2[is_labeled, ])

                sem_loss = (sem_t1_loss + sem_t2_loss) / 2
                supervised_loss = change_loss + sem_loss

                if cfg.MODEL.ENABLE_SEMANTIC_CHANGE_LOSS:
                    sem_change_loss = change_criterion(logits_change_sem[is_labeled,], gt_change[is_labeled,])
                    change_sem_loss_set.append(sem_change_loss.item())
                    supervised_loss = supervised_loss + sem_change_loss

                sem_loss_set.append(sem_loss.item())
                change_loss_set.append(change_loss.item())

            if not is_labeled.all():
                is_not_labeled = torch.logical_not(is_labeled)
                n_notlabeled += torch.sum(is_not_labeled).item()

                if cfg.CONSISTENCY_TRAINER.LOSS_TYPE == 'L2':
                    y_pred_change = torch.sigmoid(logits_change)
                    consistency_loss = change_consistency_criterion(y_pred_change[is_not_labeled,],
                                                                    y_pred_change_sem[is_not_labeled,])
                else:
                    consistency_loss = change_consistency_criterion(logits_change[is_not_labeled,],
                                                                    y_pred_change_sem[is_not_labeled,])
                consistency_loss = consistency_loss * cfg.CONSISTENCY_TRAINER.LOSS_FACTOR
                consistency_loss_set.append(consistency_loss.item())

            if supervised_loss is None and consistency_loss is not None:
                loss = consistency_loss
            elif supervised_loss is not None and consistency_loss is not None:
                loss = supervised_loss + consistency_loss
            else:
                loss = supervised_loss

            loss.backward()
            optimizer.step()

            loss_set.append(loss.item())

            global_step += 1
            epoch_float = global_step / steps_per_epoch

            if global_step % cfg.LOG_FREQ == 0:
                print(f'Logging step {global_step} (epoch {epoch_float:.2f}).')

                # evaluation on sample of training and validation set
                evaluation.model_evaluation(net, cfg, device, 'training', epoch_float, global_step, enable_sem=True)
                evaluation.model_evaluation(net, cfg, device, 'validation', epoch_float, global_step, enable_sem=True)

                # logging
                time = timeit.default_timer() - start
                wandb.log({
                    'change_loss': np.mean(change_loss_set) if len(change_loss_set) > 0 else 0,
                    'change_sem_loss': np.mean(change_sem_loss_set) if len(change_sem_loss_set) > 0 else 0,
                    'sem_loss': np.mean(sem_loss_set) if len(sem_loss_set) > 0 else 0,
                    'cons_loss': np.mean(consistency_loss_set) if len(consistency_loss_set) > 0 else 0,
                    'loss': np.mean(loss_set),
                    'labeled_percentage': n_labeled / (n_labeled + n_notlabeled) * 100,
                    'time': time,
                    'step': global_step,
                    'epoch': epoch_float,
                })
                start = timeit.default_timer()
                n_labeled, n_notlabeled = 0, 0
                loss_set, sem_loss_set, change_loss_set, change_sem_loss_set, consistency_loss_set = [], [], [], [], []
            # end of batch

        assert (epoch == epoch_float)
        print(f'epoch float {epoch_float} (step {global_step}) - epoch {epoch}')
        # evaluation at the end of an epoch
        evaluation.model_evaluation(net, cfg, device, 'training', epoch_float, global_step, enable_sem=True)
        evaluation.model_evaluation(net, cfg, device, 'validation', epoch_float, global_step, enable_sem=True)
        evaluation.model_evaluation(net, cfg, device, 'test', epoch_float, global_step, enable_sem=True)

        if epoch in save_checkpoints:
            print(f'saving network', flush=True)
            networks.save_checkpoint(net, optimizer, epoch, global_step, cfg)


if __name__ == '__main__':
    args = experiment_manager.default_argument_parser().parse_known_args()[0]
    cfg = experiment_manager.setup_cfg(args)

    # make training deterministic
    torch.manual_seed(cfg.SEED)
    np.random.seed(cfg.SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    print('=== Runnning on device: p', device)

    wandb.init(
        name=cfg.NAME,
        config=cfg,
        project='siamese_ssl',
        tags=['ssl', 'cd', 'siamese', 'spacenet7', ],
        mode='online' if not cfg.DEBUG else 'disabled',
    )

    try:
        run_training(cfg)
    except KeyboardInterrupt:
        try:
            sys.exit(0)
        except SystemExit:
            os._exit(0)

NameError: name 'experiment_manager' is not defined

In [ ]:


    'sn7_train_source_L15-0331E-1257N_1327_3160_13_2018_01',
    'sn7_train_source_L15-0331E-1257N_1327_3160_13_2018_02',
    'sn7_train_source_L15-0331E-1257N_1327_3160_13_2018_03',
    'sn7_train_source_L15-0331E-1257N_1327_3160_13_2018_04',
    'sn7_train_source_L15-0331E-1257N_1327_3160_13_2018_05',
    'sn7_train_source_L15-0331E-1257N_1327_3160_13_2018_06',
    'sn7_train_source_L15-0331E-1257N_1327_3160_13_2018_07',
    'sn7_train_source_L15-0331E-1257N_1327_3160_13_2018_08',
    'sn7_train_source_L15-0331E-1257N_1327_3160_13_2018_09',
    'sn7_train_source_L15-0331E-1257N_1327_3160_13_2018_10',
    'sn7_train_source_L15-0331E-1257N_1327_3160_13_2018_11',
    'sn7_train_source_L15-0331E-1257N_1327_3160_13_2018_12',
    'sn7_train_source_L15-0331E-1257N_1327_3160_13_2019_01',
    'sn7_train_source_L15-0331E-1257N_1327_3160_13_2019_02',
    'sn7_train_source_L15-0331E-1257N_1327_3160_13_2019_03',
    'sn7_train_source_L15-0331E-1257N_1327_3160_13_2019_04',
    'sn7_train_source_L15-0331E-1257N_1327_3160_13_2019_05',
    'sn7_train_source_L15-0331E-1257N_1327_3160_13_2019_06',
    'sn7_train_source_L15-0331E-1257N_1327_3160_13_2019_07',
    'sn7_train_source_L15-0331E-1257N_1327_3160_13_2019_08',
    'sn7_train_source_L15-0331E-1257N_1327_3160_13_2019_09',
    'sn7_train_source_L15-0331E-1257N_1327_3160_13_2019_10',
    'sn7_train_source_L15-0331E-1257N_1327_3160_13_2019_11',
    'sn7_train_source_L15-0331E-1257N_1327_3160_13_2019_12',
    'sn7_train_source_L15-0331E-1257N_1327_3160_13_2020_01',
    'sn7_train_source_L15-0357E-1223N_1429_3296_13_2018_01',
    'sn7_train_source_L15-0357E-1223N_1429_3296_13_2018_02',
    'sn7_train_source_L15-0357E-1223N_1429_3296_13_2018_03',
    'sn7_train_source_L15-0357E-1223N_1429_3296_13_2018_04',
    'sn7_train_source_L15-0357E-1223N_1429_3296_13_2018_05',
    'sn7_train_source_L15-0357E-1223N_1429_3296_13_2018_06',
    'sn7_train_source_L15-0357E-1223N_1429_3296_13_2018_07',
    'sn7_train_source_L15-0357E-1223N_1429_3296_13_2018_08',
    'sn7_train_source_L15-0357E-1223N_1429_3296_13_2018_09',
    'sn7_train_source_L15-0357E-1223N_1429_3296_13_2018_10',
    'sn7_train_source_L15-0357E-1223N_1429_3296_13_2018_11',
    'sn7_train_source_L15-0357E-1223N_1429_3296_13_2018_12',
    'sn7_train_source_L15-0357E-1223N_1429_3296_13_2019_01',
    'sn7_train_source_L15-0357E-1223N_1429_3296_13_2019_02',
    'sn7_train_source_L15-0357E-1223N_1429_3296_13_2019_03',
    'sn7_train_source_L15-0357E-1223N_1429_3296_13_2019_04',
    'sn7_train_source_L15-0357E-1223N_1429_3296_13_2019_05',
    'sn7_train_source_L15-0357E-1223N_1429_3296_13_2019_06',
    'sn7_train_source_L15-0357E-1223N_1429_3296_13_2019_07',
    'sn7_train_source_L15-0357E-1223N_1429_3296_13_2019_08',
    'sn7_train_source_L15-0357E-1223N_1429_3296_13_2019_09',
    'sn7_train_source_L15-0357E-1223N_1429_3296_13_2019_10',
    'sn7_train_source_L15-0357E-1223N_1429_3296_13_2019_11',
    'sn7_train_source_L15-0357E-1223N_1429_3296_13_2019_12',
    'sn7_train_source_L15-0357E-1223N_1429_3296_13_2020_01',
    'sn7_train_source_L15-0358E-1220N_1433_3310_13_2018_02',
    'sn7_train_source_L15-0358E-1220N_1433_3310_13_2018_03',
    'sn7_train_source_L15-0358E-1220N_1433_3310_13_2018_04',
    'sn7_train_source_L15-0358E-1220N_1433_3310_13_2018_05',
    'sn7_train_source_L15-0358E-1220N_1433_3310_13_2018_06',
    'sn7_train_source_L15-0358E-1220N_1433_3310_13_2018_07',
    'sn7_train_source_L15-0358E-1220N_1433_3310_13_2018_08',
    'sn7_train_source_L15-0358E-1220N_1433_3310_13_2018_09',
    'sn7_train_source_L15-0358E-1220N_1433_3310_13_2018_10',
    'sn7_train_source_L15-0358E-1220N_1433_3310_13_2018_11',
    'sn7_train_source_L15-0358E-1220N_1433_3310_13_2018_12',
    'sn7_train_source_L15-0358E-1220N_1433_3310_13_2019_01',
    'sn7_train_source_L15-0358E-1220N_1433_3310_13_2019_02',
    'sn7_train_source_L15-0358E-1220N_1433_3310_13_2019_03',
    'sn7_train_source_L15-0358E-1220N_1433_3310_13_2019_04',
    'sn7_train_source_L15-0358E-1220N_1433_3310_13_2019_05',
    'sn7_train_source_L15-0358E-1220N_1433_3310_13_2019_06',
    'sn7_train_source_L15-0358E-1220N_1433_3310_13_2019_07',
    'sn7_train_source_L15-0358E-1220N_1433_3310_13_2019_08',
    'sn7_train_source_L15-0358E-1220N_1433_3310_13_2019_09',
    'sn7_train_source_L15-0358E-1220N_1433_3310_13_2019_10',
    'sn7_train_source_L15-0358E-1220N_1433_3310_13_2019_11',
    'sn7_train_source_L15-0358E-1220N_1433_3310_13_2019_12',
    'sn7_train_source_L15-0358E-1220N_1433_3310_13_2020_01',
    'sn7_train_source_L15-0361E-1300N_1446_2989_13_2018_01',
    'sn7_train_source_L15-0361E-1300N_1446_2989_13_2018_02',
    'sn7_train_source_L15-0361E-1300N_1446_2989_13_2018_03',
    'sn7_train_source_L15-0361E-1300N_1446_2989_13_2018_05',
    'sn7_train_source_L15-0361E-1300N_1446_2989_13_2018_06',
    'sn7_train_source_L15-0361E-1300N_1446_2989_13_2018_07',
    'sn7_train_source_L15-0361E-1300N_1446_2989_13_2018_08',
    'sn7_train_source_L15-0361E-1300N_1446_2989_13_2018_09',
    'sn7_train_source_L15-0361E-1300N_1446_2989_13_2018_10',
    'sn7_train_source_L15-0361E-1300N_1446_2989_13_2018_11',
    'sn7_train_source_L15-0361E-1300N_1446_2989_13_2018_12',
    'sn7_train_source_L15-0361E-1300N_1446_2989_13_2019_01',
    'sn7_train_source_L15-0361E-1300N_1446_2989_13_2019_02',
    'sn7_train_source_L15-0361E-1300N_1446_2989_13_2019_03',
    'sn7_train_source_L15-0361E-1300N_1446_2989_13_2019_04',
    'sn7_train_source_L15-0361E-1300N_1446_2989_13_2019_05',
    'sn7_train_source_L15-0361E-1300N_1446_2989_13_2019_06',
    'sn7_train_source_L15-0361E-1300N_1446_2989_13_2019_07',
    'sn7_train_source_L15-0361E-1300N_1446_2989_13_2019_08',
    'sn7_train_source_L15-0361E-1300N_1446_2989_13_2019_09',
    'sn7_train_source_L15-0361E-1300N_1446_2989_13_2019_10',
    'sn7_train_source_L15-0361E-1300N_1446_2989_13_2019_11',
    'sn7_train_source_L15-0361E-1300N_1446_2989_13_2019_12',
    'sn7_train_source_L15-0361E-1300N_1446_2989_13_2020_01',
    'sn7_train_source_L15-0368E-1245N_1474_3210_13_2018_01',
    'sn7_train_source_L15-0368E-1245N_1474_3210_13_2018_02',
    'sn7_train_source_L15-0368E-1245N_1474_3210_13_2018_03',
    'sn7_train_source_L15-0368E-1245N_1474_3210_13_2018_04',
    'sn7_train_source_L15-0368E-1245N_1474_3210_13_2018_06',
    'sn7_train_source_L15-0368E-1245N_1474_3210_13_2018_07',
    'sn7_train_source_L15-0368E-1245N_1474_3210_13_2018_08',
    'sn7_train_source_L15-0368E-1245N_1474_3210_13_2018_09',
    'sn7_train_source_L15-0368E-1245N_1474_3210_13_2018_10',
    'sn7_train_source_L15-0368E-1245N_1474_3210_13_2018_11',
    'sn7_train_source_L15-0368E-1245N_1474_3210_13_2018_12',
    'sn7_train_source_L15-0368E-1245N_1474_3210_13_2019_01',
    'sn7_train_source_L15-0368E-1245N_1474_3210_13_2019_02',
    'sn7_train_source_L15-0368E-1245N_1474_3210_13_2019_03',
    'sn7_train_source_L15-0368E-1245N_1474_3210_13_2019_04',
    'sn7_train_source_L15-0368E-1245N_1474_3210_13_2019_05',
    'sn7_train_source_L15-0368E-1245N_1474_3210_13_2019_06',
    'sn7_train_source_L15-0368E-1245N_1474_3210_13_2019_07',
    'sn7_train_source_L15-0368E-1245N_1474_3210_13_2019_08',
    'sn7_train_source_L15-0368E-1245N_1474_3210_13_2019_09',
    'sn7_train_source_L15-0368E-1245N_1474_3210_13_2019_10',
    'sn7_train_source_L15-0368E-1245N_1474_3210_13_2019_11',
    'sn7_train_source_L15-0368E-1245N_1474_3210_13_2019_12',
    'sn7_train_source_L15-0368E-1245N_1474_3210_13_2020_01',
    'sn7_train_source_L15-0387E-1276N_1549_3087_13_2018_01',
    'sn7_train_source_L15-0387E-1276N_1549_3087_13_2018_02',
    'sn7_train_source_L15-0387E-1276N_1549_3087_13_2018_03',
    'sn7_train_source_L15-0387E-1276N_1549_3087_13_2018_04',
    'sn7_train_source_L15-0387E-1276N_1549_3087_13_2018_05',
    'sn7_train_source_L15-0387E-1276N_1549_3087_13_2018_06',
    'sn7_train_source_L15-0387E-1276N_1549_3087_13_2018_07',
    'sn7_train_source_L15-0387E-1276N_1549_3087_13_2018_08',
    'sn7_train_source_L15-0387E-1276N_1549_3087_13_2018_09',
    'sn7_train_source_L15-0387E-1276N_1549_3087_13_2018_10',
    'sn7_train_source_L15-0387E-1276N_1549_3087_13_2018_11',
    'sn7_train_source_L15-0387E-1276N_1549_3087_13_2018_12',
    'sn7_train_source_L15-0387E-1276N_1549_3087_13_2019_03',
    'sn7_train_source_L15-0387E-1276N_1549_3087_13_2019_04',
    'sn7_train_source_L15-0387E-1276N_1549_3087_13_2019_05',
    'sn7_train_source_L15-0387E-1276N_1549_3087_13_2019_06',
    'sn7_train_source_L15-0387E-1276N_1549_3087_13_2019_07',
    'sn7_train_source_L15-0387E-1276N_1549_3087_13_2019_08',
    'sn7_train_source_L15-0387E-1276N_1549_3087_13_2019_09',
    'sn7_train_source_L15-0387E-1276N_1549_3087_13_2019_10',
    'sn7_train_source_L15-0387E-1276N_1549_3087_13_2019_11',
    'sn7_train_source_L15-0387E-1276N_1549_3087_13_2019_12',
    'sn7_train_source_L15-0434E-1218N_1736_3318_13_2018_01',
    'sn7_train_source_L15-0434E-1218N_1736_3318_13_2018_02',
    'sn7_train_source_L15-0434E-1218N_1736_3318_13_2018_03',
    'sn7_train_source_L15-0434E-1218N_1736_3318_13_2018_04',
    'sn7_train_source_L15-0434E-1218N_1736_3318_13_2018_05',
    'sn7_train_source_L15-0434E-1218N_1736_3318_13_2018_06',
    'sn7_train_source_L15-0434E-1218N_1736_3318_13_2018_07',
    'sn7_train_source_L15-0434E-1218N_1736_3318_13_2018_08',
    'sn7_train_source_L15-0434E-1218N_1736_3318_13_2018_09',
    'sn7_train_source_L15-0434E-1218N_1736_3318_13_2018_10',
    'sn7_train_source_L15-0434E-1218N_1736_3318_13_2018_11',
    'sn7_train_source_L15-0434E-1218N_1736_3318_13_2018_12',
    'sn7_train_source_L15-0434E-1218N_1736_3318_13_2019_01',
    'sn7_train_source_L15-0434E-1218N_1736_3318_13_2019_02',
    'sn7_train_source_L15-0434E-1218N_1736_3318_13_2019_03',
    'sn7_train_source_L15-0434E-1218N_1736_3318_13_2019_04',
    'sn7_train_source_L15-0434E-1218N_1736_3318_13_2019_05',
    'sn7_train_source_L15-0434E-1218N_1736_3318_13_2019_06',
    'sn7_train_source_L15-0434E-1218N_1736_3318_13_2019_07',
    'sn7_train_source_L15-0434E-1218N_1736_3318_13_2019_08',
    'sn7_train_source_L15-0434E-1218N_1736_3318_13_2019_09',
    'sn7_train_source_L15-0434E-1218N_1736_3318_13_2019_10',
    'sn7_train_source_L15-0434E-1218N_1736_3318_13_2019_11',
    'sn7_train_source_L15-0434E-1218N_1736_3318_13_2019_12',
    'sn7_train_source_L15-0434E-1218N_1736_3318_13_2020_01',
    'sn7_train_source_L15-0457E-1135N_1831_3648_13_2018_01',
    'sn7_train_source_L15-0457E-1135N_1831_3648_13_2018_02',
    'sn7_train_source_L15-0457E-1135N_1831_3648_13_2018_03',
    'sn7_train_source_L15-0457E-1135N_1831_3648_13_2018_04',
    'sn7_train_source_L15-0457E-1135N_1831_3648_13_2018_05',
    'sn7_train_source_L15-0457E-1135N_1831_3648_13_2018_06',
    'sn7_train_source_L15-0457E-1135N_1831_3648_13_2018_07',
    'sn7_train_source_L15-0457E-1135N_1831_3648_13_2018_08',
    'sn7_train_source_L15-0457E-1135N_1831_3648_13_2018_09',
    'sn7_train_source_L15-0457E-1135N_1831_3648_13_2018_10',
    'sn7_train_source_L15-0457E-1135N_1831_3648_13_2018_11',
    'sn7_train_source_L15-0457E-1135N_1831_3648_13_2018_12',
    'sn7_train_source_L15-0457E-1135N_1831_3648_13_2019_01',
    'sn7_train_source_L15-0457E-1135N_1831_3648_13_2019_02',
    'sn7_train_source_L15-0457E-1135N_1831_3648_13_2019_03',
    'sn7_train_source_L15-0457E-1135N_1831_3648_13_2019_04',
    'sn7_train_source_L15-0457E-1135N_1831_3648_13_2019_05',
    'sn7_train_source_L15-0457E-1135N_1831_3648_13_2019_06',
    'sn7_train_source_L15-0457E-1135N_1831_3648_13_2019_07',
    'sn7_train_source_L15-0457E-1135N_1831_3648_13_2019_08',
    'sn7_train_source_L15-0457E-1135N_1831_3648_13_2019_09',
    'sn7_train_source_L15-0457E-1135N_1831_3648_13_2019_10',
    'sn7_train_source_L15-0457E-1135N_1831_3648_13_2019_11',
    'sn7_train_source_L15-0457E-1135N_1831_3648_13_2019_12',
    'sn7_train_source_L15-0487E-1246N_1950_3207_13_2018_01',
    'sn7_train_source_L15-0487E-1246N_1950_3207_13_2018_02',
    'sn7_train_source_L15-0487E-1246N_1950_3207_13_2018_03',
    'sn7_train_source_L15-0487E-1246N_1950_3207_13_2018_04',
    'sn7_train_source_L15-0487E-1246N_1950_3207_13_2018_05',
    'sn7_train_source_L15-0487E-1246N_1950_3207_13_2018_06',
    'sn7_train_source_L15-0487E-1246N_1950_3207_13_2018_07',
    'sn7_train_source_L15-0487E-1246N_1950_3207_13_2018_08',
    'sn7_train_source_L15-0487E-1246N_1950_3207_13_2018_09',
    'sn7_train_source_L15-0487E-1246N_1950_3207_13_2018_10',
    'sn7_train_source_L15-0487E-1246N_1950_3207_13_2018_11',
    'sn7_train_source_L15-0487E-1246N_1950_3207_13_2018_12',
    'sn7_train_source_L15-0487E-1246N_1950_3207_13_2019_01',
    'sn7_train_source_L15-0487E-1246N_1950_3207_13_2019_02',
    'sn7_train_source_L15-0487E-1246N_1950_3207_13_2019_03',
    'sn7_train_source_L15-0487E-1246N_1950_3207_13_2019_04',
    'sn7_train_source_L15-0487E-1246N_1950_3207_13_2019_05',
    'sn7_train_source_L15-0487E-1246N_1950_3207_13_2019_06',
    'sn7_train_source_L15-0487E-1246N_1950_3207_13_2019_07',
    'sn7_train_source_L15-0487E-1246N_1950_3207_13_2019_08',
    'sn7_train_source_L15-0487E-1246N_1950_3207_13_2019_09',
    'sn7_train_source_L15-0487E-1246N_1950_3207_13_2019_10',
    'sn7_train_source_L15-0487E-1246N_1950_3207_13_2019_11',
    'sn7_train_source_L15-0487E-1246N_1950_3207_13_2019_12',
    'sn7_train_source_L15-0487E-1246N_1950_3207_13_2020_01',
    'sn7_train_source_L15-0506E-1204N_2027_3374_13_2018_01',
    'sn7_train_source_L15-0506E-1204N_2027_3374_13_2018_02',
    'sn7_train_source_L15-0506E-1204N_2027_3374_13_2018_03',
    'sn7_train_source_L15-0506E-1204N_2027_3374_13_2018_04',
    'sn7_train_source_L15-0506E-1204N_2027_3374_13_2018_05',
    'sn7_train_source_L15-0506E-1204N_2027_3374_13_2018_06',
    'sn7_train_source_L15-0506E-1204N_2027_3374_13_2018_07',
    'sn7_train_source_L15-0506E-1204N_2027_3374_13_2018_08',
    'sn7_train_source_L15-0506E-1204N_2027_3374_13_2018_09',
    'sn7_train_source_L15-0506E-1204N_2027_3374_13_2018_10',
    'sn7_train_source_L15-0506E-1204N_2027_3374_13_2018_11',
    'sn7_train_source_L15-0506E-1204N_2027_3374_13_2018_12',
    'sn7_train_source_L15-0506E-1204N_2027_3374_13_2019_01',
    'sn7_train_source_L15-0506E-1204N_2027_3374_13_2019_02',
    'sn7_train_source_L15-0506E-1204N_2027_3374_13_2019_03',
    'sn7_train_source_L15-0506E-1204N_2027_3374_13_2019_04',
    'sn7_train_source_L15-0506E-1204N_2027_3374_13_2019_05',
    'sn7_train_source_L15-0506E-1204N_2027_3374_13_2019_06',
    'sn7_train_source_L15-0506E-1204N_2027_3374_13_2019_07',
    'sn7_train_source_L15-0506E-1204N_2027_3374_13_2019_08',
    'sn7_train_source_L15-0506E-1204N_2027_3374_13_2019_09',
    'sn7_train_source_L15-0506E-1204N_2027_3374_13_2019_10',
    'sn7_train_source_L15-0506E-1204N_2027_3374_13_2019_11',
    'sn7_train_source_L15-0506E-1204N_2027_3374_13_2019_12',
    'sn7_train_source_L15-0544E-1228N_2176_3279_13_2018_01',
    'sn7_train_source_L15-0544E-1228N_2176_3279_13_2018_02',
    'sn7_train_source_L15-0544E-1228N_2176_3279_13_2018_03',
    'sn7_train_source_L15-0544E-1228N_2176_3279_13_2018_04',
    'sn7_train_source_L15-0544E-1228N_2176_3279_13_2018_05',
    'sn7_train_source_L15-0544E-1228N_2176_3279_13_2018_06',
    'sn7_train_source_L15-0544E-1228N_2176_3279_13_2018_07',
    'sn7_train_source_L15-0544E-1228N_2176_3279_13_2018_08',
    'sn7_train_source_L15-0544E-1228N_2176_3279_13_2018_09',
    'sn7_train_source_L15-0544E-1228N_2176_3279_13_2018_10',
    'sn7_train_source_L15-0544E-1228N_2176_3279_13_2018_11',
    'sn7_train_source_L15-0544E-1228N_2176_3279_13_2018_12',
    'sn7_train_source_L15-0544E-1228N_2176_3279_13_2019_01',
    'sn7_train_source_L15-0544E-1228N_2176_3279_13_2019_02',
    'sn7_train_source_L15-0544E-1228N_2176_3279_13_2019_03',
    'sn7_train_source_L15-0544E-1228N_2176_3279_13_2019_04',
    'sn7_train_source_L15-0544E-1228N_2176_3279_13_2019_05',
    'sn7_train_source_L15-0544E-1228N_2176_3279_13_2019_06',
    'sn7_train_source_L15-0544E-1228N_2176_3279_13_2019_07',
    'sn7_train_source_L15-0544E-1228N_2176_3279_13_2019_08',
    'sn7_train_source_L15-0544E-1228N_2176_3279_13_2019_09',
    'sn7_train_source_L15-0544E-1228N_2176_3279_13_2019_10',
    'sn7_train_source_L15-0544E-1228N_2176_3279_13_2019_11',
    'sn7_train_source_L15-0544E-1228N_2176_3279_13_2019_12',
    'sn7_train_source_L15-0566E-1185N_2265_3451_13_2018_01',
    'sn7_train_source_L15-0566E-1185N_2265_3451_13_2018_02',
    'sn7_train_source_L15-0566E-1185N_2265_3451_13_2018_03',
    'sn7_train_source_L15-0566E-1185N_2265_3451_13_2018_04',
    'sn7_train_source_L15-0566E-1185N_2265_3451_13_2018_05',
    'sn7_train_source_L15-0566E-1185N_2265_3451_13_2018_07',
    'sn7_train_source_L15-0566E-1185N_2265_3451_13_2018_08',
    'sn7_train_source_L15-0566E-1185N_2265_3451_13_2018_10',
    'sn7_train_source_L15-0566E-1185N_2265_3451_13_2018_11',
    'sn7_train_source_L15-0566E-1185N_2265_3451_13_2019_01',
    'sn7_train_source_L15-0566E-1185N_2265_3451_13_2019_02',
    'sn7_train_source_L15-0566E-1185N_2265_3451_13_2019_03',
    'sn7_train_source_L15-0566E-1185N_2265_3451_13_2019_04',
    'sn7_train_source_L15-0566E-1185N_2265_3451_13_2019_05',
    'sn7_train_source_L15-0566E-1185N_2265_3451_13_2019_06',
    'sn7_train_source_L15-0566E-1185N_2265_3451_13_2019_07',
    'sn7_train_source_L15-0566E-1185N_2265_3451_13_2019_09',
    'sn7_train_source_L15-0566E-1185N_2265_3451_13_2019_10',
    'sn7_train_source_L15-0566E-1185N_2265_3451_13_2019_11',
    'sn7_train_source_L15-0566E-1185N_2265_3451_13_2019_12',
    'sn7_train_source_L15-0566E-1185N_2265_3451_13_2020_01',
    'sn7_train_source_L15-0571E-1075N_2287_3888_13_2018_01',
    'sn7_train_source_L15-0571E-1075N_2287_3888_13_2018_02',
    'sn7_train_source_L15-0571E-1075N_2287_3888_13_2018_03',
    'sn7_train_source_L15-0571E-1075N_2287_3888_13_2018_04',
    'sn7_train_source_L15-0571E-1075N_2287_3888_13_2018_07',
    'sn7_train_source_L15-0571E-1075N_2287_3888_13_2018_08',
    'sn7_train_source_L15-0571E-1075N_2287_3888_13_2018_09',
    'sn7_train_source_L15-0571E-1075N_2287_3888_13_2018_10',
    'sn7_train_source_L15-0571E-1075N_2287_3888_13_2018_11',
    'sn7_train_source_L15-0571E-1075N_2287_3888_13_2018_12',
    'sn7_train_source_L15-0571E-1075N_2287_3888_13_2019_01',
    'sn7_train_source_L15-0571E-1075N_2287_3888_13_2019_02',
    'sn7_train_source_L15-0571E-1075N_2287_3888_13_2019_03',
    'sn7_train_source_L15-0571E-1075N_2287_3888_13_2019_04',
    'sn7_train_source_L15-0571E-1075N_2287_3888_13_2019_05',
    'sn7_train_source_L15-0571E-1075N_2287_3888_13_2019_06',
    'sn7_train_source_L15-0571E-1075N_2287_3888_13_2019_08',
    'sn7_train_source_L15-0571E-1075N_2287_3888_13_2019_09',
    'sn7_train_source_L15-0571E-1075N_2287_3888_13_2019_11',
    'sn7_train_source_L15-0571E-1075N_2287_3888_13_2020_01',
    'sn7_train_source_L15-0577E-1243N_2309_3217_13_2018_02',
    'sn7_train_source_L15-0577E-1243N_2309_3217_13_2018_04',
    'sn7_train_source_L15-0577E-1243N_2309_3217_13_2018_05',
    'sn7_train_source_L15-0577E-1243N_2309_3217_13_2018_06',
    'sn7_train_source_L15-0577E-1243N_2309_3217_13_2018_07',
    'sn7_train_source_L15-0577E-1243N_2309_3217_13_2018_08',
    'sn7_train_source_L15-0577E-1243N_2309_3217_13_2018_09',
    'sn7_train_source_L15-0577E-1243N_2309_3217_13_2018_10',
    'sn7_train_source_L15-0577E-1243N_2309_3217_13_2018_11',
    'sn7_train_source_L15-0577E-1243N_2309_3217_13_2018_12',
    'sn7_train_source_L15-0577E-1243N_2309_3217_13_2019_01',
    'sn7_train_source_L15-0577E-1243N_2309_3217_13_2019_02',
    'sn7_train_source_L15-0577E-1243N_2309_3217_13_2019_03',
    'sn7_train_source_L15-0577E-1243N_2309_3217_13_2019_04',
    'sn7_train_source_L15-0577E-1243N_2309_3217_13_2019_05',
    'sn7_train_source_L15-0577E-1243N_2309_3217_13_2019_06',
    'sn7_train_source_L15-0577E-1243N_2309_3217_13_2019_07',
    'sn7_train_source_L15-0577E-1243N_2309_3217_13_2019_08',
    'sn7_train_source_L15-0577E-1243N_2309_3217_13_2019_09',
    'sn7_train_source_L15-0577E-1243N_2309_3217_13_2019_10',
    'sn7_train_source_L15-0577E-1243N_2309_3217_13_2019_11',
    'sn7_train_source_L15-0586E-1127N_2345_3680_13_2018_01',
    'sn7_train_source_L15-0586E-1127N_2345_3680_13_2018_02',
    'sn7_train_source_L15-0586E-1127N_2345_3680_13_2018_03',
    'sn7_train_source_L15-0586E-1127N_2345_3680_13_2018_04',
    'sn7_train_source_L15-0586E-1127N_2345_3680_13_2018_05',
    'sn7_train_source_L15-0586E-1127N_2345_3680_13_2018_06',
    'sn7_train_source_L15-0586E-1127N_2345_3680_13_2018_07',
    'sn7_train_source_L15-0586E-1127N_2345_3680_13_2018_08',
    'sn7_train_source_L15-0586E-1127N_2345_3680_13_2018_09',
    'sn7_train_source_L15-0586E-1127N_2345_3680_13_2018_10',
    'sn7_train_source_L15-0586E-1127N_2345_3680_13_2018_11',
    'sn7_train_source_L15-0586E-1127N_2345_3680_13_2018_12',
    'sn7_train_source_L15-0586E-1127N_2345_3680_13_2019_01',
    'sn7_train_source_L15-0586E-1127N_2345_3680_13_2019_02',
    'sn7_train_source_L15-0586E-1127N_2345_3680_13_2019_03',
    'sn7_train_source_L15-0586E-1127N_2345_3680_13_2019_04',
    'sn7_train_source_L15-0586E-1127N_2345_3680_13_2019_05',
    'sn7_train_source_L15-0586E-1127N_2345_3680_13_2019_06',
    'sn7_train_source_L15-0586E-1127N_2345_3680_13_2019_07',
    'sn7_train_source_L15-0586E-1127N_2345_3680_13_2019_08',
    'sn7_train_source_L15-0586E-1127N_2345_3680_13_2019_09',
    'sn7_train_source_L15-0586E-1127N_2345_3680_13_2019_10',
    'sn7_train_source_L15-0586E-1127N_2345_3680_13_2019_11',
    'sn7_train_source_L15-0586E-1127N_2345_3680_13_2019_12',
    'sn7_train_source_L15-0586E-1127N_2345_3680_13_2020_01',
    'sn7_train_source_L15-0595E-1278N_2383_3079_13_2018_01',
    'sn7_train_source_L15-0595E-1278N_2383_3079_13_2018_02',
    'sn7_train_source_L15-0595E-1278N_2383_3079_13_2018_03',
    'sn7_train_source_L15-0595E-1278N_2383_3079_13_2018_04',
    'sn7_train_source_L15-0595E-1278N_2383_3079_13_2018_05',
    'sn7_train_source_L15-0595E-1278N_2383_3079_13_2018_06',
    'sn7_train_source_L15-0595E-1278N_2383_3079_13_2018_07',
    'sn7_train_source_L15-0595E-1278N_2383_3079_13_2018_08',
    'sn7_train_source_L15-0595E-1278N_2383_3079_13_2018_09',
    'sn7_train_source_L15-0595E-1278N_2383_3079_13_2018_10',
    'sn7_train_source_L15-0595E-1278N_2383_3079_13_2018_11',
    'sn7_train_source_L15-0595E-1278N_2383_3079_13_2018_12',
    'sn7_train_source_L15-0595E-1278N_2383_3079_13_2019_01',
    'sn7_train_source_L15-0595E-1278N_2383_3079_13_2019_02',
    'sn7_train_source_L15-0595E-1278N_2383_3079_13_2019_03',
    'sn7_train_source_L15-0595E-1278N_2383_3079_13_2019_04',
    'sn7_train_source_L15-0595E-1278N_2383_3079_13_2019_05',
    'sn7_train_source_L15-0595E-1278N_2383_3079_13_2019_06',
    'sn7_train_source_L15-0595E-1278N_2383_3079_13_2019_07',
    'sn7_train_source_L15-0595E-1278N_2383_3079_13_2019_08',
    'sn7_train_source_L15-0595E-1278N_2383_3079_13_2019_09',
    'sn7_train_source_L15-0595E-1278N_2383_3079_13_2019_10',
    'sn7_train_source_L15-0595E-1278N_2383_3079_13_2019_11',
    'sn7_train_source_L15-0595E-1278N_2383_3079_13_2019_12',
    'sn7_train_source_L15-0595E-1278N_2383_3079_13_2020_01',
    'sn7_train_source_L15-0614E-0946N_2459_4406_13_2018_03',
    'sn7_train_source_L15-0614E-0946N_2459_4406_13_2018_04',
    'sn7_train_source_L15-0614E-0946N_2459_4406_13_2018_05',
    'sn7_train_source_L15-0614E-0946N_2459_4406_13_2018_06',
    'sn7_train_source_L15-0614E-0946N_2459_4406_13_2018_07',
    'sn7_train_source_L15-0614E-0946N_2459_4406_13_2018_08',
    'sn7_train_source_L15-0614E-0946N_2459_4406_13_2018_09',
    'sn7_train_source_L15-0614E-0946N_2459_4406_13_2018_11',
    'sn7_train_source_L15-0614E-0946N_2459_4406_13_2018_12',
    'sn7_train_source_L15-0614E-0946N_2459_4406_13_2019_01',
    'sn7_train_source_L15-0614E-0946N_2459_4406_13_2019_02',
    'sn7_train_source_L15-0614E-0946N_2459_4406_13_2019_03',
    'sn7_train_source_L15-0614E-0946N_2459_4406_13_2019_04',
    'sn7_train_source_L15-0614E-0946N_2459_4406_13_2019_05',
    'sn7_train_source_L15-0614E-0946N_2459_4406_13_2019_06',
    'sn7_train_source_L15-0614E-0946N_2459_4406_13_2019_07',
    'sn7_train_source_L15-0614E-0946N_2459_4406_13_2019_08',
    'sn7_train_source_L15-0614E-0946N_2459_4406_13_2019_09',
    'sn7_train_source_L15-0614E-0946N_2459_4406_13_2019_10',
    'sn7_train_source_L15-0614E-0946N_2459_4406_13_2019_11',
    'sn7_train_source_L15-0614E-0946N_2459_4406_13_2020_01',
    'sn7_train_source_L15-0632E-0892N_2528_4620_13_2018_01',
    'sn7_train_source_L15-0632E-0892N_2528_4620_13_2018_02',
    'sn7_train_source_L15-0632E-0892N_2528_4620_13_2018_03',
    'sn7_train_source_L15-0632E-0892N_2528_4620_13_2018_04',
    'sn7_train_source_L15-0632E-0892N_2528_4620_13_2018_05',
    'sn7_train_source_L15-0632E-0892N_2528_4620_13_2018_06',
    'sn7_train_source_L15-0632E-0892N_2528_4620_13_2018_07',
    'sn7_train_source_L15-0632E-0892N_2528_4620_13_2018_08',
    'sn7_train_source_L15-0632E-0892N_2528_4620_13_2018_09',
    'sn7_train_source_L15-0632E-0892N_2528_4620_13_2018_10',
    'sn7_train_source_L15-0632E-0892N_2528_4620_13_2018_11',
    'sn7_train_source_L15-0632E-0892N_2528_4620_13_2018_12',
    'sn7_train_source_L15-0632E-0892N_2528_4620_13_2019_01',
    'sn7_train_source_L15-0632E-0892N_2528_4620_13_2019_02',
    'sn7_train_source_L15-0632E-0892N_2528_4620_13_2019_03',
    'sn7_train_source_L15-0632E-0892N_2528_4620_13_2019_04',
    'sn7_train_source_L15-0632E-0892N_2528_4620_13_2019_05',
    'sn7_train_source_L15-0632E-0892N_2528_4620_13_2019_06',
    'sn7_train_source_L15-0632E-0892N_2528_4620_13_2019_07',
    'sn7_train_source_L15-0632E-0892N_2528_4620_13_2019_08',
    'sn7_train_source_L15-0632E-0892N_2528_4620_13_2019_09',
    'sn7_train_source_L15-0632E-0892N_2528_4620_13_2019_10',
    'sn7_train_source_L15-0632E-0892N_2528_4620_13_2019_11',
    'sn7_train_source_L15-0632E-0892N_2528_4620_13_2019_12',
    'sn7_train_source_L15-0632E-0892N_2528_4620_13_2020_01',
    'sn7_train_source_L15-0683E-1006N_2732_4164_13_2018_01',
    'sn7_train_source_L15-0683E-1006N_2732_4164_13_2018_02',
    'sn7_train_source_L15-0683E-1006N_2732_4164_13_2018_03',
    'sn7_train_source_L15-0683E-1006N_2732_4164_13_2018_04',
    'sn7_train_source_L15-0683E-1006N_2732_4164_13_2018_06',
    'sn7_train_source_L15-0683E-1006N_2732_4164_13_2018_07',
    'sn7_train_source_L15-0683E-1006N_2732_4164_13_2018_08',
    'sn7_train_source_L15-0683E-1006N_2732_4164_13_2018_09',
    'sn7_train_source_L15-0683E-1006N_2732_4164_13_2018_10',
    'sn7_train_source_L15-0683E-1006N_2732_4164_13_2018_11',
    'sn7_train_source_L15-0683E-1006N_2732_4164_13_2019_01',
    'sn7_train_source_L15-0683E-1006N_2732_4164_13_2019_03',
    'sn7_train_source_L15-0683E-1006N_2732_4164_13_2019_04',
    'sn7_train_source_L15-0683E-1006N_2732_4164_13_2019_05',
    'sn7_train_source_L15-0683E-1006N_2732_4164_13_2019_06',
    'sn7_train_source_L15-0683E-1006N_2732_4164_13_2019_07',
    'sn7_train_source_L15-0683E-1006N_2732_4164_13_2019_08',
    'sn7_train_source_L15-0683E-1006N_2732_4164_13_2019_09',
    'sn7_train_source_L15-0683E-1006N_2732_4164_13_2019_10',
    'sn7_train_source_L15-0683E-1006N_2732_4164_13_2019_11',
    'sn7_train_source_L15-0683E-1006N_2732_4164_13_2019_12',
    'sn7_train_source_L15-0683E-1006N_2732_4164_13_2020_01',
    'sn7_train_source_L15-0760E-0887N_3041_4643_13_2018_01',
    'sn7_train_source_L15-0760E-0887N_3041_4643_13_2018_02',
    'sn7_train_source_L15-0760E-0887N_3041_4643_13_2018_03',
    'sn7_train_source_L15-0760E-0887N_3041_4643_13_2018_04',
    'sn7_train_source_L15-0760E-0887N_3041_4643_13_2018_05',
    'sn7_train_source_L15-0760E-0887N_3041_4643_13_2018_06',
    'sn7_train_source_L15-0760E-0887N_3041_4643_13_2018_07',
    'sn7_train_source_L15-0760E-0887N_3041_4643_13_2018_08',
    'sn7_train_source_L15-0760E-0887N_3041_4643_13_2018_09',
    'sn7_train_source_L15-0760E-0887N_3041_4643_13_2018_10',
    'sn7_train_source_L15-0760E-0887N_3041_4643_13_2018_11',
    'sn7_train_source_L15-0760E-0887N_3041_4643_13_2018_12',
    'sn7_train_source_L15-0760E-0887N_3041_4643_13_2019_01',
    'sn7_train_source_L15-0760E-0887N_3041_4643_13_2019_02',
    'sn7_train_source_L15-0760E-0887N_3041_4643_13_2019_03',
    'sn7_train_source_L15-0760E-0887N_3041_4643_13_2019_04',
    'sn7_train_source_L15-0760E-0887N_3041_4643_13_2019_05',
    'sn7_train_source_L15-0760E-0887N_3041_4643_13_2019_06',
    'sn7_train_source_L15-0760E-0887N_3041_4643_13_2019_07',
    'sn7_train_source_L15-0760E-0887N_3041_4643_13_2019_08',
    'sn7_train_source_L15-0760E-0887N_3041_4643_13_2019_09',
    'sn7_train_source_L15-0760E-0887N_3041_4643_13_2019_10',
    'sn7_train_source_L15-0760E-0887N_3041_4643_13_2019_11',
    'sn7_train_source_L15-0760E-0887N_3041_4643_13_2019_12',
    'sn7_train_source_L15-0760E-0887N_3041_4643_13_2020_01',
    'sn7_train_source_L15-0924E-1108N_3699_3757_13_2018_01',
    'sn7_train_source_L15-0924E-1108N_3699_3757_13_2018_02',
    'sn7_train_source_L15-0924E-1108N_3699_3757_13_2018_03',
    'sn7_train_source_L15-0924E-1108N_3699_3757_13_2018_04',
    'sn7_train_source_L15-0924E-1108N_3699_3757_13_2018_05',
    'sn7_train_source_L15-0924E-1108N_3699_3757_13_2018_06',
    'sn7_train_source_L15-0924E-1108N_3699_3757_13_2018_07',
    'sn7_train_source_L15-0924E-1108N_3699_3757_13_2018_09',
    'sn7_train_source_L15-0924E-1108N_3699_3757_13_2018_10',
    'sn7_train_source_L15-0924E-1108N_3699_3757_13_2018_11',
    'sn7_train_source_L15-0924E-1108N_3699_3757_13_2018_12',
    'sn7_train_source_L15-0924E-1108N_3699_3757_13_2019_01',
    'sn7_train_source_L15-0924E-1108N_3699_3757_13_2019_02',
    'sn7_train_source_L15-0924E-1108N_3699_3757_13_2019_03',
    'sn7_train_source_L15-0924E-1108N_3699_3757_13_2019_04',
    'sn7_train_source_L15-0924E-1108N_3699_3757_13_2019_05',
    'sn7_train_source_L15-0924E-1108N_3699_3757_13_2019_06',
    'sn7_train_source_L15-0924E-1108N_3699_3757_13_2019_07',
    'sn7_train_source_L15-0924E-1108N_3699_3757_13_2019_08',
    'sn7_train_source_L15-0924E-1108N_3699_3757_13_2019_10',
    'sn7_train_source_L15-0924E-1108N_3699_3757_13_2019_11',
    'sn7_train_source_L15-0924E-1108N_3699_3757_13_2019_12',
    'sn7_train_source_L15-0924E-1108N_3699_3757_13_2020_01',
    'sn7_train_source_L15-0977E-1187N_3911_3441_13_2018_01',
    'sn7_train_source_L15-0977E-1187N_3911_3441_13_2018_02',
    'sn7_train_source_L15-0977E-1187N_3911_3441_13_2018_03',
    'sn7_train_source_L15-0977E-1187N_3911_3441_13_2018_04',
    'sn7_train_source_L15-0977E-1187N_3911_3441_13_2018_05',
    'sn7_train_source_L15-0977E-1187N_3911_3441_13_2018_06',
    'sn7_train_source_L15-0977E-1187N_3911_3441_13_2018_07',
    'sn7_train_source_L15-0977E-1187N_3911_3441_13_2018_08',
    'sn7_train_source_L15-0977E-1187N_3911_3441_13_2018_09',
    'sn7_train_source_L15-0977E-1187N_3911_3441_13_2018_10',
    'sn7_train_source_L15-0977E-1187N_3911_3441_13_2018_11',
    'sn7_train_source_L15-0977E-1187N_3911_3441_13_2018_12',
    'sn7_train_source_L15-0977E-1187N_3911_3441_13_2019_01',
    'sn7_train_source_L15-0977E-1187N_3911_3441_13_2019_02',
    'sn7_train_source_L15-0977E-1187N_3911_3441_13_2019_03',
    'sn7_train_source_L15-0977E-1187N_3911_3441_13_2019_04',
    'sn7_train_source_L15-0977E-1187N_3911_3441_13_2019_05',
    'sn7_train_source_L15-0977E-1187N_3911_3441_13_2019_06',
    'sn7_train_source_L15-0977E-1187N_3911_3441_13_2019_07',
    'sn7_train_source_L15-0977E-1187N_3911_3441_13_2019_08',
    'sn7_train_source_L15-0977E-1187N_3911_3441_13_2019_09',
    'sn7_train_source_L15-0977E-1187N_3911_3441_13_2019_10',
    'sn7_train_source_L15-0977E-1187N_3911_3441_13_2019_11',
    'sn7_train_source_L15-0977E-1187N_3911_3441_13_2019_12',
    'sn7_train_source_L15-0977E-1187N_3911_3441_13_2020_01',
    'sn7_train_source_L15-1014E-1375N_4056_2688_13_2018_01',
    'sn7_train_source_L15-1014E-1375N_4056_2688_13_2018_02',
    'sn7_train_source_L15-1014E-1375N_4056_2688_13_2018_03',
    'sn7_train_source_L15-1014E-1375N_4056_2688_13_2018_04',
    'sn7_train_source_L15-1014E-1375N_4056_2688_13_2018_05',
    'sn7_train_source_L15-1014E-1375N_4056_2688_13_2018_06',
    'sn7_train_source_L15-1014E-1375N_4056_2688_13_2018_07',
    'sn7_train_source_L15-1014E-1375N_4056_2688_13_2018_08',
    'sn7_train_source_L15-1014E-1375N_4056_2688_13_2018_09',
    'sn7_train_source_L15-1014E-1375N_4056_2688_13_2018_10',
    'sn7_train_source_L15-1014E-1375N_4056_2688_13_2018_11',
    'sn7_train_source_L15-1014E-1375N_4056_2688_13_2018_12',
    'sn7_train_source_L15-1014E-1375N_4056_2688_13_2019_01',
    'sn7_train_source_L15-1014E-1375N_4056_2688_13_2019_02',
    'sn7_train_source_L15-1014E-1375N_4056_2688_13_2019_03',
    'sn7_train_source_L15-1014E-1375N_4056_2688_13_2019_04',
    'sn7_train_source_L15-1014E-1375N_4056_2688_13_2019_05',
    'sn7_train_source_L15-1014E-1375N_4056_2688_13_2019_06',
    'sn7_train_source_L15-1014E-1375N_4056_2688_13_2019_07',
    'sn7_train_source_L15-1014E-1375N_4056_2688_13_2019_08',
    'sn7_train_source_L15-1014E-1375N_4056_2688_13_2019_09',
    'sn7_train_source_L15-1014E-1375N_4056_2688_13_2019_10',
    'sn7_train_source_L15-1014E-1375N_4056_2688_13_2019_12',
    'sn7_train_source_L15-1014E-1375N_4056_2688_13_2020_01',
    'sn7_train_source_L15-1015E-1062N_4061_3941_13_2018_01',
    'sn7_train_source_L15-1015E-1062N_4061_3941_13_2018_02',
    'sn7_train_source_L15-1015E-1062N_4061_3941_13_2018_03',
    'sn7_train_source_L15-1015E-1062N_4061_3941_13_2018_04',
    'sn7_train_source_L15-1015E-1062N_4061_3941_13_2018_05',
    'sn7_train_source_L15-1015E-1062N_4061_3941_13_2018_06',
    'sn7_train_source_L15-1015E-1062N_4061_3941_13_2018_10',
    'sn7_train_source_L15-1015E-1062N_4061_3941_13_2018_11',
    'sn7_train_source_L15-1015E-1062N_4061_3941_13_2018_12',
    'sn7_train_source_L15-1015E-1062N_4061_3941_13_2019_01',
    'sn7_train_source_L15-1015E-1062N_4061_3941_13_2019_02',
    'sn7_train_source_L15-1015E-1062N_4061_3941_13_2019_03',
    'sn7_train_source_L15-1015E-1062N_4061_3941_13_2019_04',
    'sn7_train_source_L15-1015E-1062N_4061_3941_13_2019_05',
    'sn7_train_source_L15-1015E-1062N_4061_3941_13_2019_10',
    'sn7_train_source_L15-1015E-1062N_4061_3941_13_2019_11',
    'sn7_train_source_L15-1015E-1062N_4061_3941_13_2019_12',
    'sn7_train_source_L15-1015E-1062N_4061_3941_13_2020_01',
    'sn7_train_source_L15-1025E-1366N_4102_2726_13_2018_01',
    'sn7_train_source_L15-1025E-1366N_4102_2726_13_2018_02',
    'sn7_train_source_L15-1025E-1366N_4102_2726_13_2018_03',
    'sn7_train_source_L15-1025E-1366N_4102_2726_13_2018_04',
    'sn7_train_source_L15-1025E-1366N_4102_2726_13_2018_05',
    'sn7_train_source_L15-1025E-1366N_4102_2726_13_2018_06',
    'sn7_train_source_L15-1025E-1366N_4102_2726_13_2018_07',
    'sn7_train_source_L15-1025E-1366N_4102_2726_13_2018_08',
    'sn7_train_source_L15-1025E-1366N_4102_2726_13_2018_09',
    'sn7_train_source_L15-1025E-1366N_4102_2726_13_2018_10',
    'sn7_train_source_L15-1025E-1366N_4102_2726_13_2018_11',
    'sn7_train_source_L15-1025E-1366N_4102_2726_13_2018_12',
    'sn7_train_source_L15-1025E-1366N_4102_2726_13_2019_01',
    'sn7_train_source_L15-1025E-1366N_4102_2726_13_2019_03',
    'sn7_train_source_L15-1025E-1366N_4102_2726_13_2019_04',
    'sn7_train_source_L15-1025E-1366N_4102_2726_13_2019_05',
    'sn7_train_source_L15-1025E-1366N_4102_2726_13_2019_06',
    'sn7_train_source_L15-1025E-1366N_4102_2726_13_2019_07',
    'sn7_train_source_L15-1025E-1366N_4102_2726_13_2019_08',
    'sn7_train_source_L15-1025E-1366N_4102_2726_13_2019_09',
    'sn7_train_source_L15-1025E-1366N_4102_2726_13_2019_10',
    'sn7_train_source_L15-1025E-1366N_4102_2726_13_2019_11',
    'sn7_train_source_L15-1025E-1366N_4102_2726_13_2019_12',
    'sn7_train_source_L15-1025E-1366N_4102_2726_13_2020_01',
    'sn7_train_source_L15-1049E-1370N_4196_2710_13_2018_01',
    'sn7_train_source_L15-1049E-1370N_4196_2710_13_2018_02',
    'sn7_train_source_L15-1049E-1370N_4196_2710_13_2018_03',
    'sn7_train_source_L15-1049E-1370N_4196_2710_13_2018_04',
    'sn7_train_source_L15-1049E-1370N_4196_2710_13_2018_05',
    'sn7_train_source_L15-1049E-1370N_4196_2710_13_2018_06',
    'sn7_train_source_L15-1049E-1370N_4196_2710_13_2018_07',
    'sn7_train_source_L15-1049E-1370N_4196_2710_13_2018_08',
    'sn7_train_source_L15-1049E-1370N_4196_2710_13_2018_09',
    'sn7_train_source_L15-1049E-1370N_4196_2710_13_2018_10',
    'sn7_train_source_L15-1049E-1370N_4196_2710_13_2018_11',
    'sn7_train_source_L15-1049E-1370N_4196_2710_13_2018_12',
    'sn7_train_source_L15-1049E-1370N_4196_2710_13_2019_01',
    'sn7_train_source_L15-1049E-1370N_4196_2710_13_2019_02',
    'sn7_train_source_L15-1049E-1370N_4196_2710_13_2019_03',
    'sn7_train_source_L15-1049E-1370N_4196_2710_13_2019_04',
    'sn7_train_source_L15-1049E-1370N_4196_2710_13_2019_05',
    'sn7_train_source_L15-1049E-1370N_4196_2710_13_2019_06',
    'sn7_train_source_L15-1049E-1370N_4196_2710_13_2019_07',
    'sn7_train_source_L15-1049E-1370N_4196_2710_13_2019_08',
    'sn7_train_source_L15-1049E-1370N_4196_2710_13_2019_09',
    'sn7_train_source_L15-1049E-1370N_4196_2710_13_2019_10',
    'sn7_train_source_L15-1049E-1370N_4196_2710_13_2019_11',
    'sn7_train_source_L15-1049E-1370N_4196_2710_13_2019_12',
    'sn7_train_source_L15-1049E-1370N_4196_2710_13_2020_01',
    'sn7_train_source_L15-1138E-1216N_4553_3325_13_2018_01',
    'sn7_train_source_L15-1138E-1216N_4553_3325_13_2018_02',
    'sn7_train_source_L15-1138E-1216N_4553_3325_13_2018_03',
    'sn7_train_source_L15-1138E-1216N_4553_3325_13_2018_04',
    'sn7_train_source_L15-1138E-1216N_4553_3325_13_2018_05',
    'sn7_train_source_L15-1138E-1216N_4553_3325_13_2018_06',
    'sn7_train_source_L15-1138E-1216N_4553_3325_13_2018_07',
    'sn7_train_source_L15-1138E-1216N_4553_3325_13_2018_08',
    'sn7_train_source_L15-1138E-1216N_4553_3325_13_2018_09',
    'sn7_train_source_L15-1138E-1216N_4553_3325_13_2018_10',
    'sn7_train_source_L15-1138E-1216N_4553_3325_13_2018_11',
    'sn7_train_source_L15-1138E-1216N_4553_3325_13_2018_12',
    'sn7_train_source_L15-1138E-1216N_4553_3325_13_2019_01',
    'sn7_train_source_L15-1138E-1216N_4553_3325_13_2019_02',
    'sn7_train_source_L15-1138E-1216N_4553_3325_13_2019_03',
    'sn7_train_source_L15-1138E-1216N_4553_3325_13_2019_04',
    'sn7_train_source_L15-1138E-1216N_4553_3325_13_2019_05',
    'sn7_train_source_L15-1138E-1216N_4553_3325_13_2019_06',
    'sn7_train_source_L15-1138E-1216N_4553_3325_13_2019_07',
    'sn7_train_source_L15-1138E-1216N_4553_3325_13_2019_08',
    'sn7_train_source_L15-1138E-1216N_4553_3325_13_2019_09',
    'sn7_train_source_L15-1138E-1216N_4553_3325_13_2019_10',
    'sn7_train_source_L15-1138E-1216N_4553_3325_13_2019_11',
    'sn7_train_source_L15-1138E-1216N_4553_3325_13_2019_12',
    'sn7_train_source_L15-1138E-1216N_4553_3325_13_2020_01',
    'sn7_train_source_L15-1172E-1306N_4688_2967_13_2018_01',
    'sn7_train_source_L15-1172E-1306N_4688_2967_13_2018_02',
    'sn7_train_source_L15-1172E-1306N_4688_2967_13_2018_03',
    'sn7_train_source_L15-1172E-1306N_4688_2967_13_2018_04',
    'sn7_train_source_L15-1172E-1306N_4688_2967_13_2018_05',
    'sn7_train_source_L15-1172E-1306N_4688_2967_13_2018_06',
    'sn7_train_source_L15-1172E-1306N_4688_2967_13_2018_07',
    'sn7_train_source_L15-1172E-1306N_4688_2967_13_2018_08',
    'sn7_train_source_L15-1172E-1306N_4688_2967_13_2018_09',
    'sn7_train_source_L15-1172E-1306N_4688_2967_13_2018_10',
    'sn7_train_source_L15-1172E-1306N_4688_2967_13_2018_11',
    'sn7_train_source_L15-1172E-1306N_4688_2967_13_2018_12',
    'sn7_train_source_L15-1172E-1306N_4688_2967_13_2019_02',
    'sn7_train_source_L15-1172E-1306N_4688_2967_13_2019_03',
    'sn7_train_source_L15-1172E-1306N_4688_2967_13_2019_04',
    'sn7_train_source_L15-1172E-1306N_4688_2967_13_2019_05',
    'sn7_train_source_L15-1172E-1306N_4688_2967_13_2019_06',
    'sn7_train_source_L15-1172E-1306N_4688_2967_13_2019_07',
    'sn7_train_source_L15-1172E-1306N_4688_2967_13_2019_08',
    'sn7_train_source_L15-1172E-1306N_4688_2967_13_2019_09',
    'sn7_train_source_L15-1172E-1306N_4688_2967_13_2019_10',
    'sn7_train_source_L15-1172E-1306N_4688_2967_13_2019_11',
    'sn7_train_source_L15-1172E-1306N_4688_2967_13_2019_12',
    'sn7_train_source_L15-1172E-1306N_4688_2967_13_2020_01',
    'sn7_train_source_L15-1185E-0935N_4742_4450_13_2018_01',
    'sn7_train_source_L15-1185E-0935N_4742_4450_13_2018_02',
    'sn7_train_source_L15-1185E-0935N_4742_4450_13_2018_04',
    'sn7_train_source_L15-1185E-0935N_4742_4450_13_2018_05',
    'sn7_train_source_L15-1185E-0935N_4742_4450_13_2018_06',
    'sn7_train_source_L15-1185E-0935N_4742_4450_13_2018_07',
    'sn7_train_source_L15-1185E-0935N_4742_4450_13_2018_08',
    'sn7_train_source_L15-1185E-0935N_4742_4450_13_2018_09',
    'sn7_train_source_L15-1185E-0935N_4742_4450_13_2018_10',
    'sn7_train_source_L15-1185E-0935N_4742_4450_13_2018_11',
    'sn7_train_source_L15-1185E-0935N_4742_4450_13_2018_12',
    'sn7_train_source_L15-1185E-0935N_4742_4450_13_2019_01',
    'sn7_train_source_L15-1185E-0935N_4742_4450_13_2019_02',
    'sn7_train_source_L15-1185E-0935N_4742_4450_13_2019_03',
    'sn7_train_source_L15-1185E-0935N_4742_4450_13_2019_04',
    'sn7_train_source_L15-1185E-0935N_4742_4450_13_2019_05',
    'sn7_train_source_L15-1185E-0935N_4742_4450_13_2019_06',
    'sn7_train_source_L15-1185E-0935N_4742_4450_13_2019_07',
    'sn7_train_source_L15-1185E-0935N_4742_4450_13_2019_08',
    'sn7_train_source_L15-1185E-0935N_4742_4450_13_2019_09',
    'sn7_train_source_L15-1185E-0935N_4742_4450_13_2019_10',
    'sn7_train_source_L15-1185E-0935N_4742_4450_13_2019_11',
    'sn7_train_source_L15-1185E-0935N_4742_4450_13_2019_12',
    'sn7_train_source_L15-1185E-0935N_4742_4450_13_2020_01',
    'sn7_train_source_L15-1200E-0847N_4802_4803_13_2018_02',
    'sn7_train_source_L15-1200E-0847N_4802_4803_13_2018_03',
    'sn7_train_source_L15-1200E-0847N_4802_4803_13_2018_04',
    'sn7_train_source_L15-1200E-0847N_4802_4803_13_2018_05',
    'sn7_train_source_L15-1200E-0847N_4802_4803_13_2018_06',
    'sn7_train_source_L15-1200E-0847N_4802_4803_13_2018_07',
    'sn7_train_source_L15-1200E-0847N_4802_4803_13_2018_08',
    'sn7_train_source_L15-1200E-0847N_4802_4803_13_2018_09',
    'sn7_train_source_L15-1200E-0847N_4802_4803_13_2018_10',
    'sn7_train_source_L15-1200E-0847N_4802_4803_13_2018_11',
    'sn7_train_source_L15-1200E-0847N_4802_4803_13_2018_12',
    'sn7_train_source_L15-1200E-0847N_4802_4803_13_2019_01',
    'sn7_train_source_L15-1200E-0847N_4802_4803_13_2019_02',
    'sn7_train_source_L15-1200E-0847N_4802_4803_13_2019_03',
    'sn7_train_source_L15-1200E-0847N_4802_4803_13_2019_04',
    'sn7_train_source_L15-1200E-0847N_4802_4803_13_2019_05',
    'sn7_train_source_L15-1200E-0847N_4802_4803_13_2019_06',
    'sn7_train_source_L15-1200E-0847N_4802_4803_13_2019_07',
    'sn7_train_source_L15-1200E-0847N_4802_4803_13_2019_08',
    'sn7_train_source_L15-1200E-0847N_4802_4803_13_2019_09',
    'sn7_train_source_L15-1200E-0847N_4802_4803_13_2019_10',
    'sn7_train_source_L15-1200E-0847N_4802_4803_13_2019_11',
    'sn7_train_source_L15-1200E-0847N_4802_4803_13_2019_12',
    'sn7_train_source_L15-1200E-0847N_4802_4803_13_2020_01',
    'sn7_train_source_L15-1203E-1203N_4815_3378_13_2017_07',
    'sn7_train_source_L15-1203E-1203N_4815_3378_13_2017_08',
    'sn7_train_source_L15-1203E-1203N_4815_3378_13_2017_09',
    'sn7_train_source_L15-1203E-1203N_4815_3378_13_2017_10',
    'sn7_train_source_L15-1203E-1203N_4815_3378_13_2017_11',
    'sn7_train_source_L15-1203E-1203N_4815_3378_13_2018_01',
    'sn7_train_source_L15-1203E-1203N_4815_3378_13_2018_02',
    'sn7_train_source_L15-1203E-1203N_4815_3378_13_2018_03',
    'sn7_train_source_L15-1203E-1203N_4815_3378_13_2018_04',
    'sn7_train_source_L15-1203E-1203N_4815_3378_13_2018_05',
    'sn7_train_source_L15-1203E-1203N_4815_3378_13_2018_06',
    'sn7_train_source_L15-1203E-1203N_4815_3378_13_2018_07',
    'sn7_train_source_L15-1203E-1203N_4815_3378_13_2018_08',
    'sn7_train_source_L15-1203E-1203N_4815_3378_13_2018_09',
    'sn7_train_source_L15-1203E-1203N_4815_3378_13_2018_10',
    'sn7_train_source_L15-1203E-1203N_4815_3378_13_2018_11',
    'sn7_train_source_L15-1203E-1203N_4815_3378_13_2018_12',
    'sn7_train_source_L15-1203E-1203N_4815_3378_13_2019_01',
    'sn7_train_source_L15-1203E-1203N_4815_3378_13_2019_02',
    'sn7_train_source_L15-1203E-1203N_4815_3378_13_2019_03',
    'sn7_train_source_L15-1203E-1203N_4815_3378_13_2019_04',
    'sn7_train_source_L15-1203E-1203N_4815_3378_13_2019_05',
    'sn7_train_source_L15-1203E-1203N_4815_3378_13_2019_06',
    'sn7_train_source_L15-1203E-1203N_4815_3378_13_2019_07',
    'sn7_train_source_L15-1203E-1203N_4815_3378_13_2019_08',
    'sn7_train_source_L15-1203E-1203N_4815_3378_13_2019_09',
    'sn7_train_source_L15-1204E-1202N_4816_3380_13_2017_07',
    'sn7_train_source_L15-1204E-1202N_4816_3380_13_2017_08',
    'sn7_train_source_L15-1204E-1202N_4816_3380_13_2017_09',
    'sn7_train_source_L15-1204E-1202N_4816_3380_13_2017_10',
    'sn7_train_source_L15-1204E-1202N_4816_3380_13_2017_11',
    'sn7_train_source_L15-1204E-1202N_4816_3380_13_2018_01',
    'sn7_train_source_L15-1204E-1202N_4816_3380_13_2018_02',
    'sn7_train_source_L15-1204E-1202N_4816_3380_13_2018_03',
    'sn7_train_source_L15-1204E-1202N_4816_3380_13_2018_04',
    'sn7_train_source_L15-1204E-1202N_4816_3380_13_2018_05',
    'sn7_train_source_L15-1204E-1202N_4816_3380_13_2018_06',
    'sn7_train_source_L15-1204E-1202N_4816_3380_13_2018_07',
    'sn7_train_source_L15-1204E-1202N_4816_3380_13_2018_08',
    'sn7_train_source_L15-1204E-1202N_4816_3380_13_2018_09',
    'sn7_train_source_L15-1204E-1202N_4816_3380_13_2018_10',
    'sn7_train_source_L15-1204E-1202N_4816_3380_13_2018_11',
    'sn7_train_source_L15-1204E-1202N_4816_3380_13_2018_12',
    'sn7_train_source_L15-1204E-1202N_4816_3380_13_2019_01',
    'sn7_train_source_L15-1204E-1202N_4816_3380_13_2019_02',
    'sn7_train_source_L15-1204E-1202N_4816_3380_13_2019_03',
    'sn7_train_source_L15-1204E-1202N_4816_3380_13_2019_04',
    'sn7_train_source_L15-1204E-1202N_4816_3380_13_2019_05',
    'sn7_train_source_L15-1204E-1202N_4816_3380_13_2019_06',
    'sn7_train_source_L15-1204E-1202N_4816_3380_13_2019_07',
    'sn7_train_source_L15-1204E-1202N_4816_3380_13_2019_08',
    'sn7_train_source_L15-1204E-1202N_4816_3380_13_2019_09',
    'sn7_train_source_L15-1204E-1204N_4819_3372_13_2018_01',
    'sn7_train_source_L15-1204E-1204N_4819_3372_13_2018_02',
    'sn7_train_source_L15-1204E-1204N_4819_3372_13_2018_03',
    'sn7_train_source_L15-1204E-1204N_4819_3372_13_2018_04',
    'sn7_train_source_L15-1204E-1204N_4819_3372_13_2018_05',
    'sn7_train_source_L15-1204E-1204N_4819_3372_13_2018_06',
    'sn7_train_source_L15-1204E-1204N_4819_3372_13_2018_07',
    'sn7_train_source_L15-1204E-1204N_4819_3372_13_2018_08',
    'sn7_train_source_L15-1204E-1204N_4819_3372_13_2018_09',
    'sn7_train_source_L15-1204E-1204N_4819_3372_13_2018_10',
    'sn7_train_source_L15-1204E-1204N_4819_3372_13_2018_11',
    'sn7_train_source_L15-1204E-1204N_4819_3372_13_2018_12',
    'sn7_train_source_L15-1204E-1204N_4819_3372_13_2019_01',
    'sn7_train_source_L15-1204E-1204N_4819_3372_13_2019_02',
    'sn7_train_source_L15-1204E-1204N_4819_3372_13_2019_03',
    'sn7_train_source_L15-1204E-1204N_4819_3372_13_2019_04',
    'sn7_train_source_L15-1204E-1204N_4819_3372_13_2019_05',
    'sn7_train_source_L15-1204E-1204N_4819_3372_13_2019_06',
    'sn7_train_source_L15-1204E-1204N_4819_3372_13_2019_07',
    'sn7_train_source_L15-1204E-1204N_4819_3372_13_2019_08',
    'sn7_train_source_L15-1204E-1204N_4819_3372_13_2019_09',
    'sn7_train_source_L15-1204E-1204N_4819_3372_13_2019_10',
    'sn7_train_source_L15-1204E-1204N_4819_3372_13_2019_11',
    'sn7_train_source_L15-1204E-1204N_4819_3372_13_2019_12',
    'sn7_train_source_L15-1204E-1204N_4819_3372_13_2020_01',
    'sn7_train_source_L15-1209E-1113N_4838_3737_13_2018_01',
    'sn7_train_source_L15-1209E-1113N_4838_3737_13_2018_02',
    'sn7_train_source_L15-1209E-1113N_4838_3737_13_2018_03',
    'sn7_train_source_L15-1209E-1113N_4838_3737_13_2018_04',
    'sn7_train_source_L15-1209E-1113N_4838_3737_13_2018_05',
    'sn7_train_source_L15-1209E-1113N_4838_3737_13_2018_06',
    'sn7_train_source_L15-1209E-1113N_4838_3737_13_2018_07',
    'sn7_train_source_L15-1209E-1113N_4838_3737_13_2018_08',
    'sn7_train_source_L15-1209E-1113N_4838_3737_13_2018_09',
    'sn7_train_source_L15-1209E-1113N_4838_3737_13_2018_10',
    'sn7_train_source_L15-1209E-1113N_4838_3737_13_2018_11',
    'sn7_train_source_L15-1209E-1113N_4838_3737_13_2018_12',
    'sn7_train_source_L15-1209E-1113N_4838_3737_13_2019_01',
    'sn7_train_source_L15-1209E-1113N_4838_3737_13_2019_02',
    'sn7_train_source_L15-1209E-1113N_4838_3737_13_2019_03',
    'sn7_train_source_L15-1209E-1113N_4838_3737_13_2019_04',
    'sn7_train_source_L15-1209E-1113N_4838_3737_13_2019_05',
    'sn7_train_source_L15-1209E-1113N_4838_3737_13_2019_06',
    'sn7_train_source_L15-1209E-1113N_4838_3737_13_2019_07',
    'sn7_train_source_L15-1209E-1113N_4838_3737_13_2019_08',
    'sn7_train_source_L15-1209E-1113N_4838_3737_13_2019_09',
    'sn7_train_source_L15-1209E-1113N_4838_3737_13_2019_10',
    'sn7_train_source_L15-1209E-1113N_4838_3737_13_2019_11',
    'sn7_train_source_L15-1209E-1113N_4838_3737_13_2019_12',
    'sn7_train_source_L15-1209E-1113N_4838_3737_13_2020_01',
    'sn7_train_source_L15-1210E-1025N_4840_4088_13_2018_01',
    'sn7_train_source_L15-1210E-1025N_4840_4088_13_2018_02',
    'sn7_train_source_L15-1210E-1025N_4840_4088_13_2018_03',
    'sn7_train_source_L15-1210E-1025N_4840_4088_13_2018_04',
    'sn7_train_source_L15-1210E-1025N_4840_4088_13_2018_06',
    'sn7_train_source_L15-1210E-1025N_4840_4088_13_2018_07',
    'sn7_train_source_L15-1210E-1025N_4840_4088_13_2018_08',
    'sn7_train_source_L15-1210E-1025N_4840_4088_13_2018_09',
    'sn7_train_source_L15-1210E-1025N_4840_4088_13_2018_10',
    'sn7_train_source_L15-1210E-1025N_4840_4088_13_2018_11',
    'sn7_train_source_L15-1210E-1025N_4840_4088_13_2018_12',
    'sn7_train_source_L15-1210E-1025N_4840_4088_13_2019_01',
    'sn7_train_source_L15-1210E-1025N_4840_4088_13_2019_02',
    'sn7_train_source_L15-1210E-1025N_4840_4088_13_2019_03',
    'sn7_train_source_L15-1210E-1025N_4840_4088_13_2019_04',
    'sn7_train_source_L15-1210E-1025N_4840_4088_13_2019_05',
    'sn7_train_source_L15-1210E-1025N_4840_4088_13_2019_06',
    'sn7_train_source_L15-1210E-1025N_4840_4088_13_2019_07',
    'sn7_train_source_L15-1210E-1025N_4840_4088_13_2019_08',
    'sn7_train_source_L15-1210E-1025N_4840_4088_13_2019_09',
    'sn7_train_source_L15-1210E-1025N_4840_4088_13_2019_11',
    'sn7_train_source_L15-1210E-1025N_4840_4088_13_2019_12',
    'sn7_train_source_L15-1210E-1025N_4840_4088_13_2020_01',
    'sn7_train_source_L15-1276E-1107N_5105_3761_13_2018_01',
    'sn7_train_source_L15-1276E-1107N_5105_3761_13_2018_02',
    'sn7_train_source_L15-1276E-1107N_5105_3761_13_2018_03',
    'sn7_train_source_L15-1276E-1107N_5105_3761_13_2018_04',
    'sn7_train_source_L15-1276E-1107N_5105_3761_13_2018_05',
    'sn7_train_source_L15-1276E-1107N_5105_3761_13_2018_06',
    'sn7_train_source_L15-1276E-1107N_5105_3761_13_2018_07',
    'sn7_train_source_L15-1276E-1107N_5105_3761_13_2018_08',
    'sn7_train_source_L15-1276E-1107N_5105_3761_13_2018_09',
    'sn7_train_source_L15-1276E-1107N_5105_3761_13_2018_10',
    'sn7_train_source_L15-1276E-1107N_5105_3761_13_2018_11',
    'sn7_train_source_L15-1276E-1107N_5105_3761_13_2018_12',
    'sn7_train_source_L15-1276E-1107N_5105_3761_13_2019_01',
    'sn7_train_source_L15-1276E-1107N_5105_3761_13_2019_02',
    'sn7_train_source_L15-1276E-1107N_5105_3761_13_2019_03',
    'sn7_train_source_L15-1276E-1107N_5105_3761_13_2019_04',
    'sn7_train_source_L15-1276E-1107N_5105_3761_13_2019_05',
    'sn7_train_source_L15-1276E-1107N_5105_3761_13_2019_06',
    'sn7_train_source_L15-1276E-1107N_5105_3761_13_2019_07',
    'sn7_train_source_L15-1276E-1107N_5105_3761_13_2019_08',
    'sn7_train_source_L15-1276E-1107N_5105_3761_13_2019_09',
    'sn7_train_source_L15-1276E-1107N_5105_3761_13_2019_10',
    'sn7_train_source_L15-1276E-1107N_5105_3761_13_2019_11',
    'sn7_train_source_L15-1276E-1107N_5105_3761_13_2019_12',
    'sn7_train_source_L15-1276E-1107N_5105_3761_13_2020_01',
    'sn7_train_source_L15-1289E-1169N_5156_3514_13_2018_01',
    'sn7_train_source_L15-1289E-1169N_5156_3514_13_2018_02',
    'sn7_train_source_L15-1289E-1169N_5156_3514_13_2018_03',
    'sn7_train_source_L15-1289E-1169N_5156_3514_13_2018_04',
    'sn7_train_source_L15-1289E-1169N_5156_3514_13_2018_05',
    'sn7_train_source_L15-1289E-1169N_5156_3514_13_2018_06',
    'sn7_train_source_L15-1289E-1169N_5156_3514_13_2018_07',
    'sn7_train_source_L15-1289E-1169N_5156_3514_13_2018_08',
    'sn7_train_source_L15-1289E-1169N_5156_3514_13_2018_09',
    'sn7_train_source_L15-1289E-1169N_5156_3514_13_2018_10',
    'sn7_train_source_L15-1289E-1169N_5156_3514_13_2018_11',
    'sn7_train_source_L15-1289E-1169N_5156_3514_13_2018_12',
    'sn7_train_source_L15-1289E-1169N_5156_3514_13_2019_01',
    'sn7_train_source_L15-1289E-1169N_5156_3514_13_2019_02',
    'sn7_train_source_L15-1289E-1169N_5156_3514_13_2019_03',
    'sn7_train_source_L15-1289E-1169N_5156_3514_13_2019_04',
    'sn7_train_source_L15-1289E-1169N_5156_3514_13_2019_05',
    'sn7_train_source_L15-1289E-1169N_5156_3514_13_2019_06',
    'sn7_train_source_L15-1289E-1169N_5156_3514_13_2019_07',
    'sn7_train_source_L15-1289E-1169N_5156_3514_13_2019_08',
    'sn7_train_source_L15-1289E-1169N_5156_3514_13_2019_09',
    'sn7_train_source_L15-1289E-1169N_5156_3514_13_2019_10',
    'sn7_train_source_L15-1289E-1169N_5156_3514_13_2019_11',
    'sn7_train_source_L15-1289E-1169N_5156_3514_13_2019_12',
    'sn7_train_source_L15-1289E-1169N_5156_3514_13_2020_01',
    'sn7_train_source_L15-1296E-1198N_5184_3399_13_2018_01',
    'sn7_train_source_L15-1296E-1198N_5184_3399_13_2018_02',
    'sn7_train_source_L15-1296E-1198N_5184_3399_13_2018_03',
    'sn7_train_source_L15-1296E-1198N_5184_3399_13_2018_04',
    'sn7_train_source_L15-1296E-1198N_5184_3399_13_2018_05',
    'sn7_train_source_L15-1296E-1198N_5184_3399_13_2018_06',
    'sn7_train_source_L15-1296E-1198N_5184_3399_13_2018_07',
    'sn7_train_source_L15-1296E-1198N_5184_3399_13_2018_08',
    'sn7_train_source_L15-1296E-1198N_5184_3399_13_2018_09',
    'sn7_train_source_L15-1296E-1198N_5184_3399_13_2018_10',
    'sn7_train_source_L15-1296E-1198N_5184_3399_13_2018_11',
    'sn7_train_source_L15-1296E-1198N_5184_3399_13_2018_12',
    'sn7_train_source_L15-1296E-1198N_5184_3399_13_2019_01',
    'sn7_train_source_L15-1296E-1198N_5184_3399_13_2019_02',
    'sn7_train_source_L15-1296E-1198N_5184_3399_13_2019_03',
    'sn7_train_source_L15-1296E-1198N_5184_3399_13_2019_04',
    'sn7_train_source_L15-1296E-1198N_5184_3399_13_2019_05',
    'sn7_train_source_L15-1296E-1198N_5184_3399_13_2019_06',
    'sn7_train_source_L15-1296E-1198N_5184_3399_13_2019_07',
    'sn7_train_source_L15-1296E-1198N_5184_3399_13_2019_08',
    'sn7_train_source_L15-1296E-1198N_5184_3399_13_2019_09',
    'sn7_train_source_L15-1296E-1198N_5184_3399_13_2019_10',
    'sn7_train_source_L15-1296E-1198N_5184_3399_13_2019_11',
    'sn7_train_source_L15-1296E-1198N_5184_3399_13_2019_12',
    'sn7_train_source_L15-1296E-1198N_5184_3399_13_2020_01',
    'sn7_train_source_L15-1298E-1322N_5193_2903_13_2018_01',
    'sn7_train_source_L15-1298E-1322N_5193_2903_13_2018_02',
    'sn7_train_source_L15-1298E-1322N_5193_2903_13_2018_03',
    'sn7_train_source_L15-1298E-1322N_5193_2903_13_2018_04',
    'sn7_train_source_L15-1298E-1322N_5193_2903_13_2018_05',
    'sn7_train_source_L15-1298E-1322N_5193_2903_13_2018_06',
    'sn7_train_source_L15-1298E-1322N_5193_2903_13_2018_07',
    'sn7_train_source_L15-1298E-1322N_5193_2903_13_2018_08',
    'sn7_train_source_L15-1298E-1322N_5193_2903_13_2018_09',
    'sn7_train_source_L15-1298E-1322N_5193_2903_13_2018_10',
    'sn7_train_source_L15-1298E-1322N_5193_2903_13_2018_11',
    'sn7_train_source_L15-1298E-1322N_5193_2903_13_2019_01',
    'sn7_train_source_L15-1298E-1322N_5193_2903_13_2019_02',
    'sn7_train_source_L15-1298E-1322N_5193_2903_13_2019_03',
    'sn7_train_source_L15-1298E-1322N_5193_2903_13_2019_04',
    'sn7_train_source_L15-1298E-1322N_5193_2903_13_2019_05',
    'sn7_train_source_L15-1298E-1322N_5193_2903_13_2019_06',
    'sn7_train_source_L15-1298E-1322N_5193_2903_13_2019_07',
    'sn7_train_source_L15-1298E-1322N_5193_2903_13_2019_08',
    'sn7_train_source_L15-1298E-1322N_5193_2903_13_2019_09',
    'sn7_train_source_L15-1298E-1322N_5193_2903_13_2019_10',
    'sn7_train_source_L15-1298E-1322N_5193_2903_13_2019_11',
    'sn7_train_source_L15-1298E-1322N_5193_2903_13_2019_12',
    'sn7_train_source_L15-1298E-1322N_5193_2903_13_2020_01',
    'sn7_train_source_L15-1335E-1166N_5342_3524_13_2018_01',
    'sn7_train_source_L15-1335E-1166N_5342_3524_13_2018_02',
    'sn7_train_source_L15-1335E-1166N_5342_3524_13_2018_03',
    'sn7_train_source_L15-1335E-1166N_5342_3524_13_2018_04',
    'sn7_train_source_L15-1335E-1166N_5342_3524_13_2018_05',
    'sn7_train_source_L15-1335E-1166N_5342_3524_13_2018_06',
    'sn7_train_source_L15-1335E-1166N_5342_3524_13_2018_07',
    'sn7_train_source_L15-1335E-1166N_5342_3524_13_2018_08',
    'sn7_train_source_L15-1335E-1166N_5342_3524_13_2018_09',
    'sn7_train_source_L15-1335E-1166N_5342_3524_13_2018_10',
    'sn7_train_source_L15-1335E-1166N_5342_3524_13_2018_11',
    'sn7_train_source_L15-1335E-1166N_5342_3524_13_2018_12',
    'sn7_train_source_L15-1335E-1166N_5342_3524_13_2019_01',
    'sn7_train_source_L15-1335E-1166N_5342_3524_13_2019_02',
    'sn7_train_source_L15-1335E-1166N_5342_3524_13_2019_03',
    'sn7_train_source_L15-1335E-1166N_5342_3524_13_2019_04',
    'sn7_train_source_L15-1335E-1166N_5342_3524_13_2019_05',
    'sn7_train_source_L15-1335E-1166N_5342_3524_13_2019_06',
    'sn7_train_source_L15-1335E-1166N_5342_3524_13_2019_07',
    'sn7_train_source_L15-1335E-1166N_5342_3524_13_2019_08',
    'sn7_train_source_L15-1335E-1166N_5342_3524_13_2019_09',
    'sn7_train_source_L15-1335E-1166N_5342_3524_13_2019_10',
    'sn7_train_source_L15-1335E-1166N_5342_3524_13_2019_11',
    'sn7_train_source_L15-1335E-1166N_5342_3524_13_2019_12',
    'sn7_train_source_L15-1335E-1166N_5342_3524_13_2020_01',
    'sn7_train_source_L15-1389E-1284N_5557_3054_13_2018_02',
    'sn7_train_source_L15-1389E-1284N_5557_3054_13_2018_03',
    'sn7_train_source_L15-1389E-1284N_5557_3054_13_2018_04',
    'sn7_train_source_L15-1389E-1284N_5557_3054_13_2018_05',
    'sn7_train_source_L15-1389E-1284N_5557_3054_13_2018_06',
    'sn7_train_source_L15-1389E-1284N_5557_3054_13_2018_07',
    'sn7_train_source_L15-1389E-1284N_5557_3054_13_2018_08',
    'sn7_train_source_L15-1389E-1284N_5557_3054_13_2018_09',
    'sn7_train_source_L15-1389E-1284N_5557_3054_13_2018_10',
    'sn7_train_source_L15-1389E-1284N_5557_3054_13_2018_11',
    'sn7_train_source_L15-1389E-1284N_5557_3054_13_2018_12',
    'sn7_train_source_L15-1389E-1284N_5557_3054_13_2019_01',
    'sn7_train_source_L15-1389E-1284N_5557_3054_13_2019_02',
    'sn7_train_source_L15-1389E-1284N_5557_3054_13_2019_03',
    'sn7_train_source_L15-1389E-1284N_5557_3054_13_2019_04',
    'sn7_train_source_L15-1389E-1284N_5557_3054_13_2019_05',
    'sn7_train_source_L15-1389E-1284N_5557_3054_13_2019_06',
    'sn7_train_source_L15-1389E-1284N_5557_3054_13_2019_07',
    'sn7_train_source_L15-1389E-1284N_5557_3054_13_2019_08',
    'sn7_train_source_L15-1389E-1284N_5557_3054_13_2019_09',
    'sn7_train_source_L15-1389E-1284N_5557_3054_13_2019_10',
    'sn7_train_source_L15-1389E-1284N_5557_3054_13_2019_11',
    'sn7_train_source_L15-1389E-1284N_5557_3054_13_2019_12',
    'sn7_train_source_L15-1389E-1284N_5557_3054_13_2020_01',
    'sn7_train_source_L15-1438E-1134N_5753_3655_13_2018_01',
    'sn7_train_source_L15-1438E-1134N_5753_3655_13_2018_02',
    'sn7_train_source_L15-1438E-1134N_5753_3655_13_2018_03',
    'sn7_train_source_L15-1438E-1134N_5753_3655_13_2018_04',
    'sn7_train_source_L15-1438E-1134N_5753_3655_13_2018_05',
    'sn7_train_source_L15-1438E-1134N_5753_3655_13_2018_06',
    'sn7_train_source_L15-1438E-1134N_5753_3655_13_2018_08',
    'sn7_train_source_L15-1438E-1134N_5753_3655_13_2018_09',
    'sn7_train_source_L15-1438E-1134N_5753_3655_13_2018_10',
    'sn7_train_source_L15-1438E-1134N_5753_3655_13_2018_11',
    'sn7_train_source_L15-1438E-1134N_5753_3655_13_2018_12',
    'sn7_train_source_L15-1438E-1134N_5753_3655_13_2019_01',
    'sn7_train_source_L15-1438E-1134N_5753_3655_13_2019_02',
    'sn7_train_source_L15-1438E-1134N_5753_3655_13_2019_03',
    'sn7_train_source_L15-1438E-1134N_5753_3655_13_2019_04',
    'sn7_train_source_L15-1438E-1134N_5753_3655_13_2019_05',
    'sn7_train_source_L15-1438E-1134N_5753_3655_13_2019_06',
    'sn7_train_source_L15-1438E-1134N_5753_3655_13_2019_09',
    'sn7_train_source_L15-1438E-1134N_5753_3655_13_2019_10',
    'sn7_train_source_L15-1438E-1134N_5753_3655_13_2019_11',
    'sn7_train_source_L15-1438E-1134N_5753_3655_13_2019_12',
    'sn7_train_source_L15-1438E-1134N_5753_3655_13_2020_01',
    'sn7_train_source_L15-1439E-1134N_5759_3655_13_2018_02',
    'sn7_train_source_L15-1439E-1134N_5759_3655_13_2018_03',
    'sn7_train_source_L15-1439E-1134N_5759_3655_13_2018_04',
    'sn7_train_source_L15-1439E-1134N_5759_3655_13_2018_05',
    'sn7_train_source_L15-1439E-1134N_5759_3655_13_2018_06',
    'sn7_train_source_L15-1439E-1134N_5759_3655_13_2018_10',
    'sn7_train_source_L15-1439E-1134N_5759_3655_13_2018_11',
    'sn7_train_source_L15-1439E-1134N_5759_3655_13_2018_12',
    'sn7_train_source_L15-1439E-1134N_5759_3655_13_2019_01',
    'sn7_train_source_L15-1439E-1134N_5759_3655_13_2019_02',
    'sn7_train_source_L15-1439E-1134N_5759_3655_13_2019_03',
    'sn7_train_source_L15-1439E-1134N_5759_3655_13_2019_04',
    'sn7_train_source_L15-1439E-1134N_5759_3655_13_2019_05',
    'sn7_train_source_L15-1439E-1134N_5759_3655_13_2019_06',
    'sn7_train_source_L15-1439E-1134N_5759_3655_13_2019_10',
    'sn7_train_source_L15-1439E-1134N_5759_3655_13_2019_11',
    'sn7_train_source_L15-1439E-1134N_5759_3655_13_2019_12',
    'sn7_train_source_L15-1439E-1134N_5759_3655_13_2020_01',
    'sn7_train_source_L15-1479E-1101N_5916_3785_13_2018_01',
    'sn7_train_source_L15-1479E-1101N_5916_3785_13_2018_02',
    'sn7_train_source_L15-1479E-1101N_5916_3785_13_2018_03',
    'sn7_train_source_L15-1479E-1101N_5916_3785_13_2018_04',
    'sn7_train_source_L15-1479E-1101N_5916_3785_13_2018_05',
    'sn7_train_source_L15-1479E-1101N_5916_3785_13_2018_06',
    'sn7_train_source_L15-1479E-1101N_5916_3785_13_2018_07',
    'sn7_train_source_L15-1479E-1101N_5916_3785_13_2018_09',
    'sn7_train_source_L15-1479E-1101N_5916_3785_13_2018_10',
    'sn7_train_source_L15-1479E-1101N_5916_3785_13_2018_11',
    'sn7_train_source_L15-1479E-1101N_5916_3785_13_2018_12',
    'sn7_train_source_L15-1479E-1101N_5916_3785_13_2019_01',
    'sn7_train_source_L15-1479E-1101N_5916_3785_13_2019_02',
    'sn7_train_source_L15-1479E-1101N_5916_3785_13_2019_03',
    'sn7_train_source_L15-1479E-1101N_5916_3785_13_2019_04',
    'sn7_train_source_L15-1479E-1101N_5916_3785_13_2019_05',
    'sn7_train_source_L15-1479E-1101N_5916_3785_13_2019_06',
    'sn7_train_source_L15-1479E-1101N_5916_3785_13_2019_07',
    'sn7_train_source_L15-1479E-1101N_5916_3785_13_2019_08',
    'sn7_train_source_L15-1479E-1101N_5916_3785_13_2019_09',
    'sn7_train_source_L15-1479E-1101N_5916_3785_13_2019_11',
    'sn7_train_source_L15-1479E-1101N_5916_3785_13_2019_12',
    'sn7_train_source_L15-1479E-1101N_5916_3785_13_2020_01',
    'sn7_train_source_L15-1481E-1119N_5927_3715_13_2018_01',
    'sn7_train_source_L15-1481E-1119N_5927_3715_13_2018_02',
    'sn7_train_source_L15-1481E-1119N_5927_3715_13_2018_03',
    'sn7_train_source_L15-1481E-1119N_5927_3715_13_2018_04',
    'sn7_train_source_L15-1481E-1119N_5927_3715_13_2018_05',
    'sn7_train_source_L15-1481E-1119N_5927_3715_13_2018_06',
    'sn7_train_source_L15-1481E-1119N_5927_3715_13_2018_08',
    'sn7_train_source_L15-1481E-1119N_5927_3715_13_2018_09',
    'sn7_train_source_L15-1481E-1119N_5927_3715_13_2018_10',
    'sn7_train_source_L15-1481E-1119N_5927_3715_13_2018_11',
    'sn7_train_source_L15-1481E-1119N_5927_3715_13_2018_12',
    'sn7_train_source_L15-1481E-1119N_5927_3715_13_2019_01',
    'sn7_train_source_L15-1481E-1119N_5927_3715_13_2019_02',
    'sn7_train_source_L15-1481E-1119N_5927_3715_13_2019_03',
    'sn7_train_source_L15-1481E-1119N_5927_3715_13_2019_04',
    'sn7_train_source_L15-1481E-1119N_5927_3715_13_2019_05',
    'sn7_train_source_L15-1481E-1119N_5927_3715_13_2019_06',
    'sn7_train_source_L15-1481E-1119N_5927_3715_13_2019_07',
    'sn7_train_source_L15-1481E-1119N_5927_3715_13_2019_08',
    'sn7_train_source_L15-1481E-1119N_5927_3715_13_2019_09',
    'sn7_train_source_L15-1481E-1119N_5927_3715_13_2019_10',
    'sn7_train_source_L15-1481E-1119N_5927_3715_13_2019_11',
    'sn7_train_source_L15-1481E-1119N_5927_3715_13_2019_12',
    'sn7_train_source_L15-1481E-1119N_5927_3715_13_2020_01',
    'sn7_train_source_L15-1538E-1163N_6154_3539_13_2018_01',
    'sn7_train_source_L15-1538E-1163N_6154_3539_13_2018_02',
    'sn7_train_source_L15-1538E-1163N_6154_3539_13_2018_03',
    'sn7_train_source_L15-1538E-1163N_6154_3539_13_2018_04',
    'sn7_train_source_L15-1538E-1163N_6154_3539_13_2018_05',
    'sn7_train_source_L15-1538E-1163N_6154_3539_13_2018_08',
    'sn7_train_source_L15-1538E-1163N_6154_3539_13_2018_09',
    'sn7_train_source_L15-1538E-1163N_6154_3539_13_2018_10',
    'sn7_train_source_L15-1538E-1163N_6154_3539_13_2018_11',
    'sn7_train_source_L15-1538E-1163N_6154_3539_13_2018_12',
    'sn7_train_source_L15-1538E-1163N_6154_3539_13_2019_01',
    'sn7_train_source_L15-1538E-1163N_6154_3539_13_2019_02',
    'sn7_train_source_L15-1538E-1163N_6154_3539_13_2019_03',
    'sn7_train_source_L15-1538E-1163N_6154_3539_13_2019_04',
    'sn7_train_source_L15-1538E-1163N_6154_3539_13_2019_05',
    'sn7_train_source_L15-1538E-1163N_6154_3539_13_2019_08',
    'sn7_train_source_L15-1538E-1163N_6154_3539_13_2019_10',
    'sn7_train_source_L15-1538E-1163N_6154_3539_13_2019_11',
    'sn7_train_source_L15-1538E-1163N_6154_3539_13_2019_12',
    'sn7_train_source_L15-1538E-1163N_6154_3539_13_2020_01',
    'sn7_train_source_L15-1615E-1205N_6460_3370_13_2017_07',
    'sn7_train_source_L15-1615E-1205N_6460_3370_13_2017_08',
    'sn7_train_source_L15-1615E-1205N_6460_3370_13_2017_09',
    'sn7_train_source_L15-1615E-1205N_6460_3370_13_2017_11',
    'sn7_train_source_L15-1615E-1205N_6460_3370_13_2018_01',
    'sn7_train_source_L15-1615E-1205N_6460_3370_13_2018_02',
    'sn7_train_source_L15-1615E-1205N_6460_3370_13_2018_03',
    'sn7_train_source_L15-1615E-1205N_6460_3370_13_2018_04',
    'sn7_train_source_L15-1615E-1205N_6460_3370_13_2018_05',
    'sn7_train_source_L15-1615E-1205N_6460_3370_13_2018_06',
    'sn7_train_source_L15-1615E-1205N_6460_3370_13_2018_07',
    'sn7_train_source_L15-1615E-1205N_6460_3370_13_2018_08',
    'sn7_train_source_L15-1615E-1205N_6460_3370_13_2018_09',
    'sn7_train_source_L15-1615E-1205N_6460_3370_13_2018_10',
    'sn7_train_source_L15-1615E-1205N_6460_3370_13_2018_12',
    'sn7_train_source_L15-1615E-1205N_6460_3370_13_2019_01',
    'sn7_train_source_L15-1615E-1205N_6460_3370_13_2019_02',
    'sn7_train_source_L15-1615E-1205N_6460_3370_13_2019_03',
    'sn7_train_source_L15-1615E-1205N_6460_3370_13_2019_04',
    'sn7_train_source_L15-1615E-1205N_6460_3370_13_2019_05',
    'sn7_train_source_L15-1615E-1205N_6460_3370_13_2019_06',
    'sn7_train_source_L15-1615E-1205N_6460_3370_13_2019_07',
    'sn7_train_source_L15-1615E-1205N_6460_3370_13_2019_08',
    'sn7_train_source_L15-1615E-1205N_6460_3370_13_2019_09',
    'sn7_train_source_L15-1615E-1206N_6460_3366_13_2017_07',
    'sn7_train_source_L15-1615E-1206N_6460_3366_13_2017_08',
    'sn7_train_source_L15-1615E-1206N_6460_3366_13_2017_09',
    'sn7_train_source_L15-1615E-1206N_6460_3366_13_2017_10',
    'sn7_train_source_L15-1615E-1206N_6460_3366_13_2017_11',
    'sn7_train_source_L15-1615E-1206N_6460_3366_13_2018_01',
    'sn7_train_source_L15-1615E-1206N_6460_3366_13_2018_02',
    'sn7_train_source_L15-1615E-1206N_6460_3366_13_2018_03',
    'sn7_train_source_L15-1615E-1206N_6460_3366_13_2018_04',
    'sn7_train_source_L15-1615E-1206N_6460_3366_13_2018_05',
    'sn7_train_source_L15-1615E-1206N_6460_3366_13_2018_06',
    'sn7_train_source_L15-1615E-1206N_6460_3366_13_2018_07',
    'sn7_train_source_L15-1615E-1206N_6460_3366_13_2018_08',
    'sn7_train_source_L15-1615E-1206N_6460_3366_13_2018_09',
    'sn7_train_source_L15-1615E-1206N_6460_3366_13_2018_10',
    'sn7_train_source_L15-1615E-1206N_6460_3366_13_2018_11',
    'sn7_train_source_L15-1615E-1206N_6460_3366_13_2018_12',
    'sn7_train_source_L15-1615E-1206N_6460_3366_13_2019_01',
    'sn7_train_source_L15-1615E-1206N_6460_3366_13_2019_02',
    'sn7_train_source_L15-1615E-1206N_6460_3366_13_2019_03',
    'sn7_train_source_L15-1615E-1206N_6460_3366_13_2019_04',
    'sn7_train_source_L15-1615E-1206N_6460_3366_13_2019_05',
    'sn7_train_source_L15-1615E-1206N_6460_3366_13_2019_06',
    'sn7_train_source_L15-1615E-1206N_6460_3366_13_2019_07',
    'sn7_train_source_L15-1615E-1206N_6460_3366_13_2019_08',
    'sn7_train_source_L15-1615E-1206N_6460_3366_13_2019_09',
    'sn7_train_source_L15-1617E-1207N_6468_3360_13_2018_02',
    'sn7_train_source_L15-1617E-1207N_6468_3360_13_2018_03',
    'sn7_train_source_L15-1617E-1207N_6468_3360_13_2018_04',
    'sn7_train_source_L15-1617E-1207N_6468_3360_13_2018_05',
    'sn7_train_source_L15-1617E-1207N_6468_3360_13_2018_06',
    'sn7_train_source_L15-1617E-1207N_6468_3360_13_2018_07',
    'sn7_train_source_L15-1617E-1207N_6468_3360_13_2018_08',
    'sn7_train_source_L15-1617E-1207N_6468_3360_13_2018_09',
    'sn7_train_source_L15-1617E-1207N_6468_3360_13_2018_10',
    'sn7_train_source_L15-1617E-1207N_6468_3360_13_2018_11',
    'sn7_train_source_L15-1617E-1207N_6468_3360_13_2018_12',
    'sn7_train_source_L15-1617E-1207N_6468_3360_13_2019_01',
    'sn7_train_source_L15-1617E-1207N_6468_3360_13_2019_02',
    'sn7_train_source_L15-1617E-1207N_6468_3360_13_2019_03',
    'sn7_train_source_L15-1617E-1207N_6468_3360_13_2019_04',
    'sn7_train_source_L15-1617E-1207N_6468_3360_13_2019_05',
    'sn7_train_source_L15-1617E-1207N_6468_3360_13_2019_07',
    'sn7_train_source_L15-1617E-1207N_6468_3360_13_2019_08',
    'sn7_train_source_L15-1617E-1207N_6468_3360_13_2019_09',
    'sn7_train_source_L15-1617E-1207N_6468_3360_13_2019_10',
    'sn7_train_source_L15-1617E-1207N_6468_3360_13_2019_11',
    'sn7_train_source_L15-1617E-1207N_6468_3360_13_2019_12',
    'sn7_train_source_L15-1617E-1207N_6468_3360_13_2020_01',
    'sn7_train_source_L15-1669E-1153N_6678_3579_13_2018_01',
    'sn7_train_source_L15-1669E-1153N_6678_3579_13_2018_02',
    'sn7_train_source_L15-1669E-1153N_6678_3579_13_2018_03',
    'sn7_train_source_L15-1669E-1153N_6678_3579_13_2018_04',
    'sn7_train_source_L15-1669E-1153N_6678_3579_13_2018_05',
    'sn7_train_source_L15-1669E-1153N_6678_3579_13_2018_06',
    'sn7_train_source_L15-1669E-1153N_6678_3579_13_2018_07',
    'sn7_train_source_L15-1669E-1153N_6678_3579_13_2018_09',
    'sn7_train_source_L15-1669E-1153N_6678_3579_13_2018_10',
    'sn7_train_source_L15-1669E-1153N_6678_3579_13_2018_11',
    'sn7_train_source_L15-1669E-1153N_6678_3579_13_2018_12',
    'sn7_train_source_L15-1669E-1153N_6678_3579_13_2019_01',
    'sn7_train_source_L15-1669E-1153N_6678_3579_13_2019_07',
    'sn7_train_source_L15-1669E-1153N_6678_3579_13_2019_08',
    'sn7_train_source_L15-1669E-1153N_6678_3579_13_2019_09',
    'sn7_train_source_L15-1669E-1153N_6678_3579_13_2019_10',
    'sn7_train_source_L15-1669E-1153N_6678_3579_13_2019_11',
    'sn7_train_source_L15-1669E-1153N_6678_3579_13_2019_12',
    'sn7_train_source_L15-1669E-1160N_6678_3548_13_2017_07',
    'sn7_train_source_L15-1669E-1160N_6678_3548_13_2017_08',
    'sn7_train_source_L15-1669E-1160N_6678_3548_13_2017_09',
    'sn7_train_source_L15-1669E-1160N_6678_3548_13_2017_10',
    'sn7_train_source_L15-1669E-1160N_6678_3548_13_2017_11',
    'sn7_train_source_L15-1669E-1160N_6678_3548_13_2018_01',
    'sn7_train_source_L15-1669E-1160N_6678_3548_13_2018_02',
    'sn7_train_source_L15-1669E-1160N_6678_3548_13_2018_03',
    'sn7_train_source_L15-1669E-1160N_6678_3548_13_2018_04',
    'sn7_train_source_L15-1669E-1160N_6678_3548_13_2018_05',
    'sn7_train_source_L15-1669E-1160N_6678_3548_13_2018_06',
    'sn7_train_source_L15-1669E-1160N_6678_3548_13_2018_07',
    'sn7_train_source_L15-1669E-1160N_6678_3548_13_2018_08',
    'sn7_train_source_L15-1669E-1160N_6678_3548_13_2018_09',
    'sn7_train_source_L15-1669E-1160N_6678_3548_13_2018_10',
    'sn7_train_source_L15-1669E-1160N_6678_3548_13_2018_11',
    'sn7_train_source_L15-1669E-1160N_6678_3548_13_2018_12',
    'sn7_train_source_L15-1669E-1160N_6678_3548_13_2019_01',
    'sn7_train_source_L15-1669E-1160N_6678_3548_13_2019_02',
    'sn7_train_source_L15-1669E-1160N_6678_3548_13_2019_03',
    'sn7_train_source_L15-1669E-1160N_6678_3548_13_2019_04',
    'sn7_train_source_L15-1669E-1160N_6678_3548_13_2019_05',
    'sn7_train_source_L15-1669E-1160N_6678_3548_13_2019_06',
    'sn7_train_source_L15-1669E-1160N_6678_3548_13_2019_07',
    'sn7_train_source_L15-1669E-1160N_6678_3548_13_2019_08',
    'sn7_train_source_L15-1669E-1160N_6678_3548_13_2019_09',
    'sn7_train_source_L15-1669E-1160N_6679_3549_13_2017_07',
    'sn7_train_source_L15-1669E-1160N_6679_3549_13_2017_08',
    'sn7_train_source_L15-1669E-1160N_6679_3549_13_2017_09',
    'sn7_train_source_L15-1669E-1160N_6679_3549_13_2017_10',
    'sn7_train_source_L15-1669E-1160N_6679_3549_13_2017_11',
    'sn7_train_source_L15-1669E-1160N_6679_3549_13_2018_01',
    'sn7_train_source_L15-1669E-1160N_6679_3549_13_2018_02',
    'sn7_train_source_L15-1669E-1160N_6679_3549_13_2018_03',
    'sn7_train_source_L15-1669E-1160N_6679_3549_13_2018_04',
    'sn7_train_source_L15-1669E-1160N_6679_3549_13_2018_05',
    'sn7_train_source_L15-1669E-1160N_6679_3549_13_2018_06',
    'sn7_train_source_L15-1669E-1160N_6679_3549_13_2018_07',
    'sn7_train_source_L15-1669E-1160N_6679_3549_13_2018_08',
    'sn7_train_source_L15-1669E-1160N_6679_3549_13_2018_09',
    'sn7_train_source_L15-1669E-1160N_6679_3549_13_2018_10',
    'sn7_train_source_L15-1669E-1160N_6679_3549_13_2018_11',
    'sn7_train_source_L15-1669E-1160N_6679_3549_13_2018_12',
    'sn7_train_source_L15-1669E-1160N_6679_3549_13_2019_01',
    'sn7_train_source_L15-1669E-1160N_6679_3549_13_2019_02',
    'sn7_train_source_L15-1669E-1160N_6679_3549_13_2019_03',
    'sn7_train_source_L15-1669E-1160N_6679_3549_13_2019_04',
    'sn7_train_source_L15-1669E-1160N_6679_3549_13_2019_05',
    'sn7_train_source_L15-1669E-1160N_6679_3549_13_2019_06',
    'sn7_train_source_L15-1669E-1160N_6679_3549_13_2019_07',
    'sn7_train_source_L15-1669E-1160N_6679_3549_13_2019_08',
    'sn7_train_source_L15-1669E-1160N_6679_3549_13_2019_09',
    'sn7_train_source_L15-1672E-1207N_6691_3363_13_2018_01',
    'sn7_train_source_L15-1672E-1207N_6691_3363_13_2018_02',
    'sn7_train_source_L15-1672E-1207N_6691_3363_13_2018_03',
    'sn7_train_source_L15-1672E-1207N_6691_3363_13_2018_04',
    'sn7_train_source_L15-1672E-1207N_6691_3363_13_2018_05',
    'sn7_train_source_L15-1672E-1207N_6691_3363_13_2018_06',
    'sn7_train_source_L15-1672E-1207N_6691_3363_13_2018_07',
    'sn7_train_source_L15-1672E-1207N_6691_3363_13_2018_08',
    'sn7_train_source_L15-1672E-1207N_6691_3363_13_2018_09',
    'sn7_train_source_L15-1672E-1207N_6691_3363_13_2018_10',
    'sn7_train_source_L15-1672E-1207N_6691_3363_13_2018_11',
    'sn7_train_source_L15-1672E-1207N_6691_3363_13_2018_12',
    'sn7_train_source_L15-1672E-1207N_6691_3363_13_2019_01',
    'sn7_train_source_L15-1672E-1207N_6691_3363_13_2019_03',
    'sn7_train_source_L15-1672E-1207N_6691_3363_13_2019_04',
    'sn7_train_source_L15-1672E-1207N_6691_3363_13_2019_05',
    'sn7_train_source_L15-1672E-1207N_6691_3363_13_2019_06',
    'sn7_train_source_L15-1672E-1207N_6691_3363_13_2019_07',
    'sn7_train_source_L15-1672E-1207N_6691_3363_13_2019_08',
    'sn7_train_source_L15-1672E-1207N_6691_3363_13_2019_09',
    'sn7_train_source_L15-1672E-1207N_6691_3363_13_2019_10',
    'sn7_train_source_L15-1672E-1207N_6691_3363_13_2019_11',
    'sn7_train_source_L15-1672E-1207N_6691_3363_13_2019_12',
    'sn7_train_source_L15-1672E-1207N_6691_3363_13_2020_01',
    'sn7_train_source_L15-1690E-1211N_6763_3346_13_2017_07',
    'sn7_train_source_L15-1690E-1211N_6763_3346_13_2017_08',
    'sn7_train_source_L15-1690E-1211N_6763_3346_13_2017_10',
    'sn7_train_source_L15-1690E-1211N_6763_3346_13_2017_11',
    'sn7_train_source_L15-1690E-1211N_6763_3346_13_2018_01',
    'sn7_train_source_L15-1690E-1211N_6763_3346_13_2018_02',
    'sn7_train_source_L15-1690E-1211N_6763_3346_13_2018_03',
    'sn7_train_source_L15-1690E-1211N_6763_3346_13_2018_04',
    'sn7_train_source_L15-1690E-1211N_6763_3346_13_2018_05',
    'sn7_train_source_L15-1690E-1211N_6763_3346_13_2018_06',
    'sn7_train_source_L15-1690E-1211N_6763_3346_13_2018_07',
    'sn7_train_source_L15-1690E-1211N_6763_3346_13_2018_08',
    'sn7_train_source_L15-1690E-1211N_6763_3346_13_2018_09',
    'sn7_train_source_L15-1690E-1211N_6763_3346_13_2018_10',
    'sn7_train_source_L15-1690E-1211N_6763_3346_13_2018_11',
    'sn7_train_source_L15-1690E-1211N_6763_3346_13_2018_12',
    'sn7_train_source_L15-1690E-1211N_6763_3346_13_2019_01',
    'sn7_train_source_L15-1690E-1211N_6763_3346_13_2019_03',
    'sn7_train_source_L15-1690E-1211N_6763_3346_13_2019_04',
    'sn7_train_source_L15-1690E-1211N_6763_3346_13_2019_05',
    'sn7_train_source_L15-1690E-1211N_6763_3346_13_2019_06',
    'sn7_train_source_L15-1690E-1211N_6763_3346_13_2019_07',
    'sn7_train_source_L15-1690E-1211N_6763_3346_13_2019_08',
    'sn7_train_source_L15-1690E-1211N_6763_3346_13_2019_09',
    'sn7_train_source_L15-1691E-1211N_6764_3347_13_2017_07',
    'sn7_train_source_L15-1691E-1211N_6764_3347_13_2017_08',
    'sn7_train_source_L15-1691E-1211N_6764_3347_13_2017_10',
    'sn7_train_source_L15-1691E-1211N_6764_3347_13_2017_11',
    'sn7_train_source_L15-1691E-1211N_6764_3347_13_2018_01',
    'sn7_train_source_L15-1691E-1211N_6764_3347_13_2018_02',
    'sn7_train_source_L15-1691E-1211N_6764_3347_13_2018_03',
    'sn7_train_source_L15-1691E-1211N_6764_3347_13_2018_04',
    'sn7_train_source_L15-1691E-1211N_6764_3347_13_2018_05',
    'sn7_train_source_L15-1691E-1211N_6764_3347_13_2018_06',
    'sn7_train_source_L15-1691E-1211N_6764_3347_13_2018_07',
    'sn7_train_source_L15-1691E-1211N_6764_3347_13_2018_08',
    'sn7_train_source_L15-1691E-1211N_6764_3347_13_2018_09',
    'sn7_train_source_L15-1691E-1211N_6764_3347_13_2018_10',
    'sn7_train_source_L15-1691E-1211N_6764_3347_13_2018_11',
    'sn7_train_source_L15-1691E-1211N_6764_3347_13_2018_12',
    'sn7_train_source_L15-1691E-1211N_6764_3347_13_2019_01',
    'sn7_train_source_L15-1691E-1211N_6764_3347_13_2019_03',
    'sn7_train_source_L15-1691E-1211N_6764_3347_13_2019_04',
    'sn7_train_source_L15-1691E-1211N_6764_3347_13_2019_05',
    'sn7_train_source_L15-1691E-1211N_6764_3347_13_2019_06',
    'sn7_train_source_L15-1691E-1211N_6764_3347_13_2019_07',
    'sn7_train_source_L15-1691E-1211N_6764_3347_13_2019_08',
    'sn7_train_source_L15-1691E-1211N_6764_3347_13_2019_09',
    'sn7_train_source_L15-1703E-1219N_6813_3313_13_2018_01',
    'sn7_train_source_L15-1703E-1219N_6813_3313_13_2018_02',
    'sn7_train_source_L15-1703E-1219N_6813_3313_13_2018_03',
    'sn7_train_source_L15-1703E-1219N_6813_3313_13_2018_04',
    'sn7_train_source_L15-1703E-1219N_6813_3313_13_2018_05',
    'sn7_train_source_L15-1703E-1219N_6813_3313_13_2018_06',
    'sn7_train_source_L15-1703E-1219N_6813_3313_13_2018_07',
    'sn7_train_source_L15-1703E-1219N_6813_3313_13_2018_08',
    'sn7_train_source_L15-1703E-1219N_6813_3313_13_2018_09',
    'sn7_train_source_L15-1703E-1219N_6813_3313_13_2018_10',
    'sn7_train_source_L15-1703E-1219N_6813_3313_13_2018_11',
    'sn7_train_source_L15-1703E-1219N_6813_3313_13_2018_12',
    'sn7_train_source_L15-1703E-1219N_6813_3313_13_2019_01',
    'sn7_train_source_L15-1703E-1219N_6813_3313_13_2019_02',
    'sn7_train_source_L15-1703E-1219N_6813_3313_13_2019_03',
    'sn7_train_source_L15-1703E-1219N_6813_3313_13_2019_04',
    'sn7_train_source_L15-1703E-1219N_6813_3313_13_2019_05',
    'sn7_train_source_L15-1703E-1219N_6813_3313_13_2019_06',
    'sn7_train_source_L15-1703E-1219N_6813_3313_13_2019_07',
    'sn7_train_source_L15-1703E-1219N_6813_3313_13_2019_08',
    'sn7_train_source_L15-1703E-1219N_6813_3313_13_2019_09',
    'sn7_train_source_L15-1703E-1219N_6813_3313_13_2019_10',
    'sn7_train_source_L15-1703E-1219N_6813_3313_13_2019_11',
    'sn7_train_source_L15-1703E-1219N_6813_3313_13_2019_12',
    'sn7_train_source_L15-1703E-1219N_6813_3313_13_2020_01',
    'sn7_train_source_L15-1709E-1112N_6838_3742_13_2018_01',
    'sn7_train_source_L15-1709E-1112N_6838_3742_13_2018_02',
    'sn7_train_source_L15-1709E-1112N_6838_3742_13_2018_03',
    'sn7_train_source_L15-1709E-1112N_6838_3742_13_2018_04',
    'sn7_train_source_L15-1709E-1112N_6838_3742_13_2018_05',
    'sn7_train_source_L15-1709E-1112N_6838_3742_13_2018_06',
    'sn7_train_source_L15-1709E-1112N_6838_3742_13_2018_07',
    'sn7_train_source_L15-1709E-1112N_6838_3742_13_2018_09',
    'sn7_train_source_L15-1709E-1112N_6838_3742_13_2018_10',
    'sn7_train_source_L15-1709E-1112N_6838_3742_13_2018_11',
    'sn7_train_source_L15-1709E-1112N_6838_3742_13_2018_12',
    'sn7_train_source_L15-1709E-1112N_6838_3742_13_2019_01',
    'sn7_train_source_L15-1709E-1112N_6838_3742_13_2019_02',
    'sn7_train_source_L15-1709E-1112N_6838_3742_13_2019_03',
    'sn7_train_source_L15-1709E-1112N_6838_3742_13_2019_04',
    'sn7_train_source_L15-1709E-1112N_6838_3742_13_2019_05',
    'sn7_train_source_L15-1709E-1112N_6838_3742_13_2019_07',
    'sn7_train_source_L15-1709E-1112N_6838_3742_13_2019_08',
    'sn7_train_source_L15-1709E-1112N_6838_3742_13_2019_10',
    'sn7_train_source_L15-1709E-1112N_6838_3742_13_2019_11',
    'sn7_train_source_L15-1709E-1112N_6838_3742_13_2019_12',
    'sn7_train_source_L15-1709E-1112N_6838_3742_13_2020_01',
    'sn7_train_source_L15-1716E-1211N_6864_3345_13_2018_01',
    'sn7_train_source_L15-1716E-1211N_6864_3345_13_2018_02',
    'sn7_train_source_L15-1716E-1211N_6864_3345_13_2018_03',
    'sn7_train_source_L15-1716E-1211N_6864_3345_13_2018_04',
    'sn7_train_source_L15-1716E-1211N_6864_3345_13_2018_05',
    'sn7_train_source_L15-1716E-1211N_6864_3345_13_2018_06',
    'sn7_train_source_L15-1716E-1211N_6864_3345_13_2018_07',
    'sn7_train_source_L15-1716E-1211N_6864_3345_13_2018_09',
    'sn7_train_source_L15-1716E-1211N_6864_3345_13_2018_10',
    'sn7_train_source_L15-1716E-1211N_6864_3345_13_2018_11',
    'sn7_train_source_L15-1716E-1211N_6864_3345_13_2018_12',
    'sn7_train_source_L15-1716E-1211N_6864_3345_13_2019_01',
    'sn7_train_source_L15-1716E-1211N_6864_3345_13_2019_02',
    'sn7_train_source_L15-1716E-1211N_6864_3345_13_2019_03',
    'sn7_train_source_L15-1716E-1211N_6864_3345_13_2019_04',
    'sn7_train_source_L15-1716E-1211N_6864_3345_13_2019_05',
    'sn7_train_source_L15-1716E-1211N_6864_3345_13_2019_06',
    'sn7_train_source_L15-1716E-1211N_6864_3345_13_2019_07',
    'sn7_train_source_L15-1716E-1211N_6864_3345_13_2019_08',
    'sn7_train_source_L15-1716E-1211N_6864_3345_13_2019_09',
    'sn7_train_source_L15-1716E-1211N_6864_3345_13_2019_10',
    'sn7_train_source_L15-1716E-1211N_6864_3345_13_2019_11',
    'sn7_train_source_L15-1716E-1211N_6864_3345_13_2019_12',
    'sn7_train_source_L15-1716E-1211N_6864_3345_13_2020_01',
    'sn7_train_source_L15-1748E-1247N_6993_3202_13_2018_01',
    'sn7_train_source_L15-1748E-1247N_6993_3202_13_2018_02',
    'sn7_train_source_L15-1748E-1247N_6993_3202_13_2018_03',
    'sn7_train_source_L15-1748E-1247N_6993_3202_13_2018_04',
    'sn7_train_source_L15-1748E-1247N_6993_3202_13_2018_05',
    'sn7_train_source_L15-1748E-1247N_6993_3202_13_2018_06',
    'sn7_train_source_L15-1748E-1247N_6993_3202_13_2018_07',
    'sn7_train_source_L15-1748E-1247N_6993_3202_13_2018_08',
    'sn7_train_source_L15-1748E-1247N_6993_3202_13_2018_09',
    'sn7_train_source_L15-1748E-1247N_6993_3202_13_2018_10',
    'sn7_train_source_L15-1748E-1247N_6993_3202_13_2018_11',
    'sn7_train_source_L15-1748E-1247N_6993_3202_13_2018_12',
    'sn7_train_source_L15-1748E-1247N_6993_3202_13_2019_01',
    'sn7_train_source_L15-1748E-1247N_6993_3202_13_2019_02',
    'sn7_train_source_L15-1748E-1247N_6993_3202_13_2019_03',
    'sn7_train_source_L15-1748E-1247N_6993_3202_13_2019_04',
    'sn7_train_source_L15-1748E-1247N_6993_3202_13_2019_05',
    'sn7_train_source_L15-1748E-1247N_6993_3202_13_2019_06',
    'sn7_train_source_L15-1748E-1247N_6993_3202_13_2019_07',
    'sn7_train_source_L15-1748E-1247N_6993_3202_13_2019_08',
    'sn7_train_source_L15-1748E-1247N_6993_3202_13_2019_09',
    'sn7_train_source_L15-1748E-1247N_6993_3202_13_2019_10',
    'sn7_train_source_L15-1748E-1247N_6993_3202_13_2019_11',
    'sn7_train_source_L15-1748E-1247N_6993_3202_13_2019_12',
    'sn7_train_source_L15-1748E-1247N_6993_3202_13_2020_01',
    'sn7_train_source_L15-1848E-0793N_7394_5018_13_2018_01',
    'sn7_train_source_L15-1848E-0793N_7394_5018_13_2018_02',
    'sn7_train_source_L15-1848E-0793N_7394_5018_13_2018_03',
    'sn7_train_source_L15-1848E-0793N_7394_5018_13_2018_04',
    'sn7_train_source_L15-1848E-0793N_7394_5018_13_2018_05',
    'sn7_train_source_L15-1848E-0793N_7394_5018_13_2018_06',
    'sn7_train_source_L15-1848E-0793N_7394_5018_13_2018_07',
    'sn7_train_source_L15-1848E-0793N_7394_5018_13_2018_08',
    'sn7_train_source_L15-1848E-0793N_7394_5018_13_2018_09',
    'sn7_train_source_L15-1848E-0793N_7394_5018_13_2018_10',
    'sn7_train_source_L15-1848E-0793N_7394_5018_13_2018_11',
    'sn7_train_source_L15-1848E-0793N_7394_5018_13_2018_12',
    'sn7_train_source_L15-1848E-0793N_7394_5018_13_2019_01',
    'sn7_train_source_L15-1848E-0793N_7394_5018_13_2019_02',
    'sn7_train_source_L15-1848E-0793N_7394_5018_13_2019_03',
    'sn7_train_source_L15-1848E-0793N_7394_5018_13_2019_04',
    'sn7_train_source_L15-1848E-0793N_7394_5018_13_2019_05',
    'sn7_train_source_L15-1848E-0793N_7394_5018_13_2019_06',
    'sn7_train_source_L15-1848E-0793N_7394_5018_13_2019_07',
    'sn7_train_source_L15-1848E-0793N_7394_5018_13_2019_08',
    'sn7_train_source_L15-1848E-0793N_7394_5018_13_2019_09',
    'sn7_train_source_L15-1848E-0793N_7394_5018_13_2019_10',
    'sn7_train_source_L15-1848E-0793N_7394_5018_13_2019_11',
    'sn7_train_source_L15-1848E-0793N_7394_5018_13_2019_12',
    'sn7_train_source_L15-1848E-0793N_7394_5018_13_2020_01',